<a href="https://colab.research.google.com/github/himamuralik/code-upload/blob/main/Ship_Trajectory_Prediction_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
%%writefile config.py
import logging
import pandas as pd

# Base directory (adjust if using Google Drive)
data_directory = '/content/data/'
# Year range
start_year = 2015
end_year = 2017
years = range(start_year, end_year + 1)


# Data source
base_url = 'https://coast.noaa.gov/htdata/CMSP/AISDataHandler/'

# Trajectory segmentation and filtering
new_trajectory_time_gap = 120 * 60  # 2 hours
sog_cutoff = 30
empirical_speed_cutoff = 40
interpolation_time_gap = 5 * 60     # 5 minutes

# History and prediction window (3h each)
length_of_history = int(3 * 60 * 60 / interpolation_time_gap) + 1
length_into_the_future = int(3 * 60 * 60 / interpolation_time_gap) - 1
min_track_length = (length_of_history + length_into_the_future) * interpolation_time_gap

# Filtering vessel types and statuses
vessel_types = [
    'cargo', 'passenger', 'fishing',
    'tug tow', 'tanker', 'pleasure craft or sailing',
]
desired_statuses = [
    'under way sailing', 'under way using engine', 'undefined'
]

# Temporal resolution (30-min only)
time_gaps = [30 * 60]

# Categorical (static) feature
categorical_columns = ['vessel_group']

# Feature columns for models
dynamic_columns = ['lat', 'lon', 'sog', 'cog', 'year', 'day', 'month']
static_columns = ['month', 'year']

# Load label mappings
try:
    statuses = pd.read_csv('navigational_statuses.csv')
    types = pd.read_csv('vessel_types.csv')
except FileNotFoundError:
    statuses = pd.DataFrame()
    types = pd.DataFrame()

# Logging
log_level = logging.WARNING

def set_log_level(level):
    global log_level
    levels = [
        logging.CRITICAL, logging.ERROR, logging.WARNING,
        logging.INFO, logging.DEBUG
    ]
    log_level = levels[level]
    logging.basicConfig(level=log_level)


Writing config.py


In [5]:
!python config.py

In [6]:
%%writefile data_config.py
class DatasetConfig:
    def __init__(self, dataset_name,
                 lat_1, lat_2, lon_1, lon_2,
                 sliding_window_movement,
                 depth_1=0, depth_2=0,
                 min_pts_to_try=None, eps_to_try=None,
                 min_pts_to_use=None, eps_to_use=None):
        self.dataset_name = dataset_name
        self.lat_1 = min(lat_1, lat_2)
        self.lat_2 = max(lat_1, lat_2)
        self.lon_1 = min(lon_1, lon_2)
        self.lon_2 = max(lon_1, lon_2)
        self.corner_1 = (lat_1, lon_1)
        self.corner_2 = (lat_2, lon_2)
        self.sliding_window_movement = sliding_window_movement
        self.min_pts_to_try = min_pts_to_try
        self.eps_to_try = eps_to_try
        self.min_pts_to_use = min_pts_to_use
        self.eps_to_use = eps_to_use
        self.depth_1 = depth_1
        self.depth_2 = depth_2

datasets = {
    'new_york': DatasetConfig(
        dataset_name='new_york',
        lat_1=39.50, lon_1=-74.50, lat_2=41.50, lon_2=-71.50,
        sliding_window_movement=60 * 60,
        min_pts_to_try=[4, 10, 20, 50, 100, 250, 500],
        eps_to_try=[
            0.0001, 0.00025, 0.0005, 0.00075,
            0.001, 0.0025, 0.005, 0.0075,
            0.01, 0.025, 0.05, 0.075,
            0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9,
            1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5,
            6, 7, 8, 9, 10
        ],
        min_pts_to_use=50,
        eps_to_use=3
    )
}


Writing data_config.py


In [7]:
!python data_config.py

In [8]:
%%writefile downloader.py
import os
import requests
import zipfile
import io
import pandas as pd
import time
import shutil  # New import for streaming
import tempfile  # New import for temporary file handling
from calendar import monthrange
from datetime import date, timedelta

import config
import data_config

# ---- Dataset Configuration ---- #
# This loads the 'new_york' dataset configuration from data_config.py
dataset_config = data_config.datasets['new_york']

# Assign geographical boundaries from the config
LAT_MIN = dataset_config.lat_1
LAT_MAX = dataset_config.lat_2
LON_MIN = dataset_config.lon_1
LON_MAX = dataset_config.lon_2
YEARS = config.years
BASE_URL = config.base_url

# ---- File Paths ---- #
# Create the base and filtered directories from the config.
BASE_DIR = os.path.join(config.data_directory, dataset_config.dataset_name, 'downloads')
FILTERED_DIR = os.path.join(BASE_DIR, 'filtered')
PROGRESS_LOG = os.path.join(BASE_DIR, 'progress_log.txt')
os.makedirs(FILTERED_DIR, exist_ok=True)

# ---- Retry-Enabled Download ---- #
def safe_request(url, retries=5, wait=10, timeout=300):
    """
    Handles downloading a file with a more robust retry mechanism.

    Tries to make a GET request and returns the response if successful (status code 200).
    It will retry a maximum of `retries` times with a `wait` period between attempts,
    and uses a longer `timeout` for large downloads.
    """
    for attempt in range(1, retries + 1):
        try:
            # Use stream=True for large downloads
            print(f" - Attempt {attempt}/{retries}: Requesting {url} with a {timeout}s timeout...")
            r = requests.get(url, stream=True, timeout=timeout)
            if r.status_code == 200:
                print(f" - Download successful on attempt {attempt}.")
                return r
            print(f" - Attempt {attempt}: HTTP {r.status_code}")
        except Exception as e:
            print(f" - Attempt {attempt}: An error occurred: {e}")
        time.sleep(wait)
    print(f" - Failed to download {url} after {retries} attempts.")
    return None

# ---- Progress Tracking ---- #
def load_completed():
    """Reads the progress log file and returns a set of completed specifiers."""
    if not os.path.exists(PROGRESS_LOG):
        return set()
    with open(PROGRESS_LOG, 'r') as f:
        return set(line.strip() for line in f)

def log_completed(specifier):
    """Appends a new specifier to the progress log file."""
    with open(PROGRESS_LOG, 'a') as f:
        f.write(specifier + '\n')

# ---- Download -> Filter -> Save ---- #
def download_and_filter(specifier):
    """
    Downloads a single file by streaming to disk, filters it, and saves it.
    """
    # specifier will be something like "2015/AIS_2015_01_18.zip"
    year_str, fname = specifier.split('/')

    # Drop the "AIS_" prefix and ".zip" when naming your parquet file.
    # The output format is already day-by-day (YYYY_MM_DD).
    datepart = fname[len("AIS_"):-len(".zip")]
    outname = os.path.join(
        FILTERED_DIR,
        f"{dataset_config.dataset_name}_{datepart}.parquet"
    )

    # Skip if the filtered file already exists on disk.
    if os.path.exists(outname):
        print(f" - Skipping {specifier}: Parquet file already exists at {outname}")
        # Log it as completed to prevent re-downloading
        log_completed(specifier)
        return

    # Construct the full URL for the download.
    url = f"{BASE_URL.rstrip('/')}/{specifier}"
    print(f"Processing {url} ...")
    r = safe_request(url)
    if not r:
        print(f" - Failed to download {specifier}")
        return

    # Use a temporary file to stream the download to disk, which is more memory-efficient.
    temp_file_path = None
    try:
        with tempfile.NamedTemporaryFile(delete=False) as temp_file:
            print(" - Streaming download to temporary file...")
            shutil.copyfileobj(r.raw, temp_file)
            temp_file_path = temp_file.name

        # Now that the file is on disk, we can process it from there.
        with zipfile.ZipFile(temp_file_path, 'r') as z:
            csv_name = next(f for f in z.namelist() if f.endswith('.csv'))
            with z.open(csv_name) as csvfile:
                # Read the CSV in chunks to handle very large files efficiently.
                chunks = pd.read_csv(
                    csvfile,
                    low_memory=False,
                    chunksize=100_000
                )

                filtered_parts = []
                for chunk in chunks:
                    # Apply the bounding-box filter using the geographical boundaries from the config.
                    sub = chunk[
                        (chunk["LAT"] >= LAT_MIN) & (chunk["LAT"] <= LAT_MAX) &
                        (chunk["LON"] >= LON_MIN) & (chunk["LON"] <= LON_MAX)
                    ]
                    if not sub.empty:
                        filtered_parts.append(sub)

        if not filtered_parts:
            print(" - No data in region, skipping.")
            # Log as completed to prevent retrying the same file repeatedly.
            log_completed(specifier)
            return

        # Concatenate all filtered chunks, sort, and save to a parquet file.
        df = pd.concat(filtered_parts, ignore_index=True)
        df = df.sort_values(["MMSI", "BaseDateTime"])
        df.to_parquet(outname, index=False, compression='snappy')
        print(f" - Saved {len(df):,} records to {outname}")

        # Record progress so we can resume later.
        log_completed(specifier)

    except Exception as e:
        print(f" - An error occurred while processing {specifier}: {e}")
    finally:
        # Always remove the temporary file, regardless of success or failure.
        if temp_file_path and os.path.exists(temp_file_path):
            os.remove(temp_file_path)

# ---- Main Loop ---- #
if __name__ == "__main__":
    # Build a list of all file specifiers for the years and months we want to download.
    specifiers = []
    for year in YEARS:
        # Since 'desired_months' is not in config.py, we'll iterate through all 12 months.
        for month in range(1, 13):
            days_in_month = monthrange(year, month)[1]
            for day in range(1, days_in_month + 1):
                # Correctly format the filename to match the NOAA server's structure (YYYY_MM_DD).
                fname = f"AIS_{year}_{month:02d}_{day:02d}.zip"
                specifiers.append(f"{year}/{fname}")

    completed_specifiers = load_completed()
    for spec in specifiers:
        # We check both the log and if the final parquet file exists to avoid re-work.
        if spec in completed_specifiers:
            print(f"Skipping {spec} (already done)")
            continue
        download_and_filter(spec)


Overwriting downloader.py


In [1]:
!python downloader.py

Skipping 2015/AIS_2015_01_01.zip (already done)
Skipping 2015/AIS_2015_01_02.zip (already done)
Skipping 2015/AIS_2015_01_03.zip (already done)
Skipping 2015/AIS_2015_01_04.zip (already done)
Skipping 2015/AIS_2015_01_05.zip (already done)
Skipping 2015/AIS_2015_01_06.zip (already done)
Processing https://coast.noaa.gov/htdata/CMSP/AISDataHandler/2015/AIS_2015_01_07.zip ...
 - Attempt 1/5: Requesting https://coast.noaa.gov/htdata/CMSP/AISDataHandler/2015/AIS_2015_01_07.zip with a 300s timeout...
 - Download successful on attempt 1.
 - Streaming download to temporary file...
 - Saved 424,905 records to /content/data/new_york/downloads/filtered/new_york_2015_01_07.parquet
Processing https://coast.noaa.gov/htdata/CMSP/AISDataHandler/2015/AIS_2015_01_08.zip ...
 - Attempt 1/5: Requesting https://coast.noaa.gov/htdata/CMSP/AISDataHandler/2015/AIS_2015_01_08.zip with a 300s timeout...
 - Download successful on attempt 1.
 - Streaming download to temporary file...
 - Saved 293,151 records to 

In [ ]:
# 1. Zip the folder
!zip -r /content/downloads_ny.zip /content/data/new_york/downloads

# 2. Mount your Google Drive
from google.colab import drive
drive.mount('/content/drive')

# 3. Copy the ZIP into MyDrive
!cp /content/downloads_ny.zip /content/drive/MyDrive/


  adding: content/data/new_york/downloads/ (stored 0%)
  adding: content/data/new_york/downloads/progress_log.txt (deflated 72%)
  adding: content/data/new_york/downloads/filtered/ (stored 0%)
  adding: content/data/new_york/downloads/filtered/new_york_2015_01_02.parquet (deflated 27%)
  adding: content/data/new_york/downloads/filtered/new_york_2015_01_06.parquet (deflated 26%)
  adding: content/data/new_york/downloads/filtered/new_york_2015_01_04.parquet (deflated 28%)
  adding: content/data/new_york/downloads/filtered/new_york_2015_01_05.parquet


zip error: Interrupted (aborting)


In [ ]:
%%writefile utils.py
import os
import re
import shutil
import urllib
import pandas as pd
import utm
from calendar import monthrange
from dateutil import rrule
from datetime import datetime

# ------------------ ZONE UTIL ------------------ #

def get_zones_from_coordinates(corner_1, corner_2):
    """Get UTM zones from lat/lon corners."""
    _, _, zone_1, _ = utm.from_latlon(*corner_1)
    _, _, zone_2, _ = utm.from_latlon(*corner_2)
    if zone_1 > 19 or zone_2 > 19:
        raise ValueError("UTM zone is outside NOAA's available data.")
    return range(min(zone_1, zone_2), max(zone_1, zone_2) + 1)

# ------------------ FILENAME PARSING ------------------ #

def get_file_specifier(year, month, zone_or_day, extension):
    """Construct file name for a given year/month/zone/day."""
    if year in (2015, 2016, 2017):
        specifier = f"AIS_{year}_{month:02d}_Zone{zone_or_day:02d}.{extension}"
    elif year >= 2018:
        specifier = f"AIS_{year}_{month:02d}_{zone_or_day:02d}.{extension}"
    else:
        raise ValueError(f"Year {year} not supported. Check NOAA AIS file format.")
    return urllib.parse.urljoin(f"{year}/", specifier)

def get_info_from_specifier(file_name):
    """Extract year, month, zone/day, extension from file path."""
    match = re.search(r'[0-9]{4}.+AIS_([0-9]{4})_([0-9]{2})_(Zone)?([0-9]{2}|\*)\.(.+)', file_name)
    if match:
        year = match.group(1)
        month = match.group(2)
        zone_or_day = match.group(4)
        extension = match.group(5)
    else:
        raise ValueError(f"Cannot parse specifier from filename: {file_name}")
    return year, month, zone_or_day, extension

def all_specifiers(zones, years, extension, dir=None):
    """Build list of all file specifiers and paths for the given zones and years."""
    specifiers = []
    paths = []

    for year in years:
        if year in (2015, 2016, 2017):
            for month in range(1, 13):
                for zone in zones:
                    spec = get_file_specifier(year, month, zone, extension)
                    specifiers.append(spec)
                    if dir:
                        paths.append(os.path.join(dir, spec))
        elif year >= 2018:
            for dt in rrule.rrule(rrule.DAILY,
                                  dtstart=datetime.strptime(f"{year}-01-01", "%Y-%m-%d"),
                                  until=datetime.strptime(f"{year}-12-31", "%Y-%m-%d")):
                spec = get_file_specifier(dt.year, dt.month, dt.day, extension)
                specifiers.append(spec)
                if dir:
                    paths.append(os.path.join(dir, spec))

    return {'specifiers': specifiers, 'paths': paths} if dir else {'specifiers': specifiers}

# ------------------ CLEANING HELPERS ------------------ #

def pd_append(values):
    """Append list of Series or scalars into a single pandas Series."""
    v1 = values[0]
    if len(values) > 2:
        rest = pd_append(values[1:])
        series = pd.concat([pd.Series([v1]) if not isinstance(v1, pd.Series) else v1, rest])
    elif len(values) == 2:
        v2 = values[1]
        series = pd.concat([
            pd.Series([v1]) if not isinstance(v1, pd.Series) else v1,
            pd.Series([v2]) if not isinstance(v2, pd.Series) else v2
        ])
    return series.reset_index(drop=True)

def to_snake_case(name):
    """Convert camelCase or PascalCase to snake_case."""
    name = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', name)
    name = re.sub('__([A-Z])', r'_\1', name)
    name = re.sub('([a-z0-9])([A-Z])', r'\1_\2', name)
    return name.lower()

# ------------------ FILE SYSTEM ------------------ #

def clear_path(path):
    """Remove file or directory if it exists."""
    if os.path.exists(path):
        if os.path.isfile(path):
            os.remove(path)
        else:
            shutil.rmtree(path)

# ------------------ TIME RANGE ------------------ #

def get_min_max_times(specifier):
    """Return start and end datetime for a given file specifier."""
    year, month, zone_or_day, _ = get_info_from_specifier(specifier)
    year = int(year)
    month = int(month)

    if year <= 2017:
        min_time = pd.to_datetime(f"{year}-{month}-01 00:00:00")
        last_day = monthrange(year, month)[1]
        max_time = pd.to_datetime(f"{year}-{month}-{last_day} 23:59:59")
    else:
        day = int(zone_or_day)
        min_time = pd.to_datetime(f"{year}-{month}-{day} 00:00:00")
        max_time = pd.to_datetime(f"{year}-{month}-{day} 23:59:59")

    return min_time, max_time


In [ ]:
!python utils.py

In [ ]:
!pip install utm

In [ ]:
!pip install haversine

In [ ]:
from google.colab import drive
import shutil
import os

# 1. Mount your Google Drive
drive.mount('/content/drive')

# 2. Define paths
#   Adjust the path below to wherever you stored downloads_ny.zip in your Drive
zip_path   = '/content/drive/MyDrive/downloads_ny.zip'
target_dir = '/content/data/new_york'  # wherever you want it on Colab

# 3. Make sure the target directory exists
os.makedirs(target_dir, exist_ok=True)

# 4. Copy the zip from Drive into your working folder
shutil.copy(zip_path, target_dir)

# 5. Unzip it in place
!unzip -o "{target_dir}/downloads_ny.zip" -d "{target_dir}"


In [ ]:
%%writefile cleaner.py
#!/usr/bin/env python3
"""
Cleaner pipeline using Dask for large-scale AIS data.

Falls back to `fastparquet` if `pyarrow` is unavailable.

Usage:
    python cleaner.py <dataset_name> --test_fraction 0.3 --validation_fraction 0.1 --log_level 3
Or run without flags and enter values interactively in Colab.
"""
import os
import glob
import re
import argparse

import numpy as np
import pandas as pd
import dask
import dask.dataframe as dd
from haversine import haversine_vector, Unit

# ── Configure Dask for lower memory usage ── #
dask.config.set({
    'dataframe.split_row_groups': True,
    'dataframe.divisions': 'sorted',
    'parquet.executor': False
})

# ── Parquet engine detection ── #
try:
    import pyarrow  # noqa: F401
    PARQUET_ENGINE = 'pyarrow'
except ImportError:
    try:
        import fastparquet  # noqa: F401
        PARQUET_ENGINE = 'fastparquet'
    except ImportError:
        raise ImportError("Install pyarrow or fastparquet to enable parquet I/O.")

import config
import data_config
from utils import clear_path, to_snake_case

# ── Helpers ── #
def clean_partition(df: pd.DataFrame) -> pd.DataFrame:
    """Per-partition cleaning: filtering, type fixes, track assignment."""
    df = df.rename(
        columns={'BaseDateTime': 'BaseDatetime', 'TranscieverClass': 'TransceiverClass'}
    )
    df.columns = [to_snake_case(c) for c in df.columns]

    if df.empty:
        return df

    # derive vessel_group from vessel_type
    if 'vessel_type' in df.columns:
        replacement_dict = {VT: G.lower().replace('/', ' or ') for _, (G, VT, _) in config.types.iterrows()}
        df['vessel_group'] = (
            df['vessel_type']
            .fillna(-1)
            .astype(int)
            .astype(str)
            .replace(replacement_dict)
        )

    df['heading'] = df['heading'].replace(511, np.nan)
    df['sog'] = df['sog'] + np.where(df['sog'] < 0, 102.6, 0)
    df['cog'] = df['cog'] + np.where(df['cog'] < 0, 409.6, 0)

    df = df[
        (df['sog'] <= config.sog_cutoff)
        & df['cog'].between(0, 360)
        & (df['heading'].isna() | df['heading'].between(0, 360))
    ]

    drop_cols = {
        'vessel_name', 'imo', 'call_sign',
        'length', 'width', 'draft', 'cargo', 'transceiver_class'
    }
    df = df.drop(columns=drop_cols & set(df.columns), errors='ignore')

    df['base_datetime'] = pd.to_datetime(df['base_datetime'])
    df = df.sort_values(['mmsi', 'base_datetime'])

    dlat = df['lat'].diff().abs()
    dlon = df['lon'].diff().abs()
    df = df[(dlat + dlon) > 0]

    df['prev_lat'] = df['lat'].shift(1)
    df['prev_lon'] = df['lon'].shift(1)
    df['prev_time'] = df['base_datetime'].shift(1)
    same = df['mmsi'] == df['mmsi'].shift(1)
    df.loc[~same, ['prev_lat', 'prev_lon', 'prev_time']] = np.nan

    pairs = df[['prev_lat', 'prev_lon', 'lat', 'lon']].dropna()
    if not pairs.empty:
        dist = haversine_vector(
            pairs[['prev_lat', 'prev_lon']].values,
            pairs[['lat', 'lon']].values,
            Unit.NAUTICAL_MILES
        )
        delta_hours = (
            df['base_datetime'] - df['prev_time']
        ).dt.total_seconds() / 3600
        df.loc[pairs.index, 'emp_speed'] = dist / delta_hours.loc[pairs.index]

    df = df[df['emp_speed'].between(0.01, config.empirical_speed_cutoff)]
    df = df.drop(columns=['prev_lat', 'prev_lon', 'prev_time', 'emp_speed'], errors='ignore')

    gap = df['base_datetime'].diff().dt.total_seconds().fillna(0)
    new_track = (
        (df['mmsi'] != df['mmsi'].shift(1))
        | (gap > config.new_trajectory_time_gap)
    )
    df['track'] = new_track.cumsum()

    rng = df.groupby('track')['base_datetime'].agg(['min', 'max'])
    valid_tracks = rng[
        (rng['max'] - rng['min']).dt.total_seconds() > config.min_track_length
    ].index
    return df[df['track'].isin(valid_tracks)]


def load_and_concat(path_or_list):
    return dd.read_parquet(
        path_or_list,
        engine=PARQUET_ENGINE,
        split_row_groups=True,
        chunksize="64MB"
    )


def month_split(files, frac):
    rx = re.compile(r".*_(\d{4})_(\d{2})_\d+\.parquet$")
    by_month = {}
    for f in files:
        m = rx.match(os.path.basename(f))
        if m:
            by_month.setdefault((m[1], m[2]), []).append(f)
    months = sorted(by_month)
    split_idx = int(len(months) * (1 - frac))
    tm, tst = months[:split_idx], months[split_idx:]
    train_files = [f for m in tm for f in by_month[m]]
    test_files = [f for m in tst for f in by_month[m]]
    return train_files, test_files


def train_valid(ddf, val_frac):
    """Split tracks into training and validation sets."""
    unique_tracks = ddf['track'].unique().compute()
    tracks = list(unique_tracks)
    np.random.seed(0)
    np.random.shuffle(tracks)
    split_idx = int(len(tracks) * (1 - val_frac))
    train_set = set(tracks[:split_idx])
    valid_set = set(tracks[split_idx:])
    return (
        ddf[ddf['track'].isin(train_set)],
        ddf[ddf['track'].isin(valid_set)]
    )


def save(ddf, out_dir, name):
    os.makedirs(out_dir, exist_ok=True)
    path = os.path.join(out_dir, f"{name}.parquet")
    clear_path(path)
    ddf.to_parquet(
        path,
        engine=PARQUET_ENGINE,
        write_metadata_file=False,
        compression='snappy'
    )
    print(f"→ Saved {name} to {path}")

# ── Main ── #
def main(dataset_name, test_frac, val_frac, loglvl):
    config.set_log_level(loglvl)
    if dataset_name not in data_config.datasets:
        print(f"Warning: '{dataset_name}' is not valid; using default.")
        dataset_name = list(data_config.datasets.keys())[0]
    ds = data_config.datasets[dataset_name]

    base = os.path.join(config.data_directory, ds.dataset_name)
    filtered_dir = os.path.join(base, 'downloads', 'filtered')
    cleaned_dir = os.path.join(base, 'cleaned')

    files = sorted(glob.glob(os.path.join(filtered_dir, f"{ds.dataset_name}_*.parquet")))
    train_files, test_files = month_split(files, test_frac)

    print("Sampling one partition for metadata...")
    sample_df = load_and_concat(test_files[:1]).partitions[0].compute()
    meta = clean_partition(sample_df)

    print(f"Cleaning TEST ({dataset_name})…")
    test_ddf = load_and_concat(test_files)
    test_clean = test_ddf.map_partitions(clean_partition, meta=meta)
    save(test_clean, cleaned_dir, 'test')

    print(f"Cleaning TRAIN ({dataset_name})…")
    train_ddf = load_and_concat(train_files)
    train_clean = train_ddf.map_partitions(clean_partition, meta=meta)
    train_set, valid_set = train_valid(train_clean, val_frac)
    save(train_set, cleaned_dir, 'train')
    save(valid_set, cleaned_dir, 'valid')


if __name__ == '__main__':
    import sys
    sys.argv = [arg for arg in sys.argv if not arg.endswith('.json')]
    parser = argparse.ArgumentParser(description='AIS Cleaner')
    parser.add_argument('dataset_name', nargs='?', default=list(data_config.datasets.keys())[0],
                        help="Which dataset to process")
    parser.add_argument('--test_fraction', type=float, default=None,
                        help="Fraction for test split (e.g. 0.3)")
    parser.add_argument('--validation_fraction', type=float, default=None,
                        help="Fraction for validation split (e.g. 0.1)")
    parser.add_argument('--log_level', type=int, choices=range(6), default=None,
                        help="Log level (0–5)")
    args, _ = parser.parse_known_args()
    if args.test_fraction is None:
        args.test_fraction = float(input("Enter test_fraction: "))
    if args.validation_fraction is None:
        args.validation_fraction = float(input("Enter validation_fraction: "))
    if args.log_level is None:
        args.log_level = int(input("Enter log_level (0–5): "))
    main(
        args.dataset_name,
        args.test_fraction,
        args.validation_fraction,
        args.log_level
    )


In [ ]:
!python cleaner.py new_york --test_fraction 0.33 --validation_fraction 0.2 --log_level 3

In [ ]:
# 1. Zip the cleaned directory
!zip -r /content/cleaned_ny.zip /content/data/new_york/cleaned

# 2. Mount Google Drive (if not already mounted)
from google.colab import drive
drive.mount('/content/drive')

# 3. Copy the ZIP into your MyDrive
!cp /content/cleaned_ny.zip /content/drive/MyDrive/


In [ ]:
import pandas as pd

# Load the Parquet part file
df = pd.read_parquet("/content/data/new_york/cleaned/test.parquet/part.0.parquet")

# Show all columns for the first 5 rows
print(df.head(5))


In [ ]:
import pandas as pd

# Load the same Parquet file (or your full cleaned set)
df = pd.read_parquet("/content/data/new_york/cleaned/test.parquet/part.0.parquet")

# 1) Are *all* headings NaN?
all_nan = df['heading'].isna().all()
print("All heading values NaN?", all_nan)

# 2) Count NaNs vs total rows
nan_count   = df['heading'].isna().sum()
total_count = len(df)
print(f"Heading NaNs: {nan_count} / {total_count} ({nan_count/total_count:.1%})")

# 3) If you want to see any valid headings:
valid_headings = df['heading'].dropna().unique()
print("Unique non-NaN headings:", valid_headings)


In [ ]:
!rm -rf /content/data/new_york/interpolated_debug

In [ ]:
%%writefile interpolator.py
#!/usr/bin/env python3
"""
Interpolator pipeline using Dask for large-scale AIS data.
Bootstraps metadata from the first partition to avoid Dask meta errors.
"""
import os
import sys
import glob
import re
import argparse

import numpy as np
import pandas as pd
import dask
import dask.dataframe as dd
from scipy.interpolate import interp1d

# Assuming these are available from the project structure
import config
import data_config
from utils import clear_path

# ------------------- Interpolation Config ------------------- #
INTERP_FUNC = np.interp
TIMESTAMP_COLUMN = 'base_datetime'
COLUMNS_TO_INTERPOLATE = ['lat', 'lon', 'sog', 'cog', 'heading']
COLUMNS_TO_USE_MOST_RECENT = []
STABLE_COLUMNS = ['mmsi', 'vessel_group', 'status']
COLUMNS_TO_CALCULATE = {'year': 'int16', 'month': 'byte'}

# ------------------- File I/O ------------------- #
PARQUET_ENGINE = 'pyarrow' if 'pyarrow' in sys.modules else 'fastparquet'

def get_paths(dataset_name, debug=False):
    """Determines source and destination paths based on dataset and debug flag."""
    base = os.path.join(config.data_directory, dataset_name)
    cleaned_dir = os.path.join(base, 'cleaned')
    cleaned_dbg = os.path.join(base, 'cleaned_debug')
    # prefer cleaned over cleaned_debug if cleaned_debug doesn't exist
    if debug and os.path.isdir(cleaned_dbg):
        src = cleaned_dbg
    else:
        src = cleaned_dir
    dst = os.path.join(base, 'interpolated_debug' if debug else 'interpolated')
    os.makedirs(dst, exist_ok=True)
    return src, dst

# ------------------- Loading ------------------- #
def load_datasets(src):
    """Loads cleaned datasets from a source directory into Dask DataFrames."""
    ds = {}
    for split in ['train', 'valid', 'test']:
        path = os.path.join(src, f"{split}.parquet")
        ds[split] = dd.read_parquet(path, engine=PARQUET_ENGINE)
        print(f"Loaded {split}: {ds[split].shape[0].compute():,} rows")
    return ds

# ------------------- Core interpolation ------------------- #
def interpolate_track(track: pd.DataFrame) -> pd.DataFrame:
    """
    Interpolates a single track's data to a new time series.
    The track DataFrame is the result of a pandas groupby operation.
    """
    if track.empty:
        return pd.DataFrame(columns=[
            TIMESTAMP_COLUMN, 'lat', 'lon', 'sog', 'cog', 'heading',
            'mmsi', 'vessel_group', 'status', 'year', 'month'
        ])

    # Convert timestamp to seconds for interpolation
    t0, t1 = track[TIMESTAMP_COLUMN].iloc[[0, -1]]
    times = np.arange(t0, t1 + 1, config.interpolation_time_gap)

    # Use interp1d to map original timestamps to new interpolated timestamps
    idx_fn = interp1d(
        track[TIMESTAMP_COLUMN], np.arange(len(track)),
        kind='previous', assume_sorted=True,
        bounds_error=False, fill_value=(0, len(track) - 1)
    )
    idx = idx_fn(times).astype(int)

    out = {TIMESTAMP_COLUMN: times}
    for col in track.columns:
        # We handle 'track' column in the outer function, so we skip it here
        if col == TIMESTAMP_COLUMN or col == 'track':
            continue
        elif col in COLUMNS_TO_INTERPOLATE:
            out[col] = INTERP_FUNC(times, track[TIMESTAMP_COLUMN], track[col])
        elif col in COLUMNS_TO_USE_MOST_RECENT:
            out[col] = track[col].iloc[idx].values
        elif col in STABLE_COLUMNS:
            out[col] = np.repeat(track[col].iat[0], len(times))
        else:
            continue

    # add year/month as separate columns
    dt = pd.to_datetime(times * 1e9)
    out['year'] = dt.year.astype('int16')
    out['month'] = dt.month.astype('byte')

    # Important: Do not return the 'track' column here. It will be part of the index.
    return pd.DataFrame(out)


def interpolate_partition(df: pd.DataFrame) -> pd.DataFrame:
    """
    Interpolates a single Dask partition by applying interpolation
    to each track. This function is mapped over each partition of the Dask DataFrame.
    """
    if df.empty:
        return df

    # Group by 'track' and apply the interpolation function.
    # This results in a MultiIndex with levels ['track', 'original_index'].
    result = df.groupby('track').apply(interpolate_track)

    # Reset the index, turning the MultiIndex levels into columns.
    # This creates columns 'track' and 'level_1'.
    result = result.reset_index()

    # Drop the spurious 'level_1' column.
    if 'level_1' in result.columns:
        result = result.drop('level_1', axis=1)

    # Set the 'track' column back as the index for the final result.
    return result.set_index('track')

# ------------------- Mapping ------------------- #
def interpolate_datasets(ds: dict) -> dict:
    """
    Applies the interpolation logic to all Dask datasets (train, valid, test).
    """
    out = {}
    for name, ddf in ds.items():
        # convert timestamp to seconds
        ddf = ddf.assign(base_datetime=(ddf['base_datetime'].astype('int64') // 10**9))
        # bootstrap meta from first partition to avoid Dask meta errors
        first = ddf.partitions[0].compute()
        meta = interpolate_partition(first)
        out[name] = ddf.map_partitions(interpolate_partition, meta=meta)
    return out

# ------------------- Saving ------------------- #
def save_datasets(ds_out: dict, dst: str) -> None:
    """Saves the interpolated Dask DataFrames to parquet files."""
    clear_path(dst)
    os.makedirs(dst, exist_ok=True)
    for name, ddf in ds_out.items():
        path = os.path.join(dst, f"{name}.parquet")
        counts = ddf.map_partitions(len, meta=('n', 'int64')).compute()
        if counts.sum() == 0:
            pd.DataFrame(columns=ddf._meta.columns).to_parquet(
                path, engine=PARQUET_ENGINE, index=False
            )
        else:
            ddf = ddf.clear_divisions()
            if PARQUET_ENGINE == 'fastparquet':
                ddf.to_parquet(path, engine=PARQUET_ENGINE, compute_divisions=False)
            else:
                ddf.to_parquet(path, engine=PARQUET_ENGINE, write_metadata_file=False)
        print(f"Saved {name} -> {path}")

# ------------------- Main ------------------- #
def main(dataset_name: str, debug: bool, loglvl: int) -> None:
    """Main function to run the interpolation pipeline."""
    config.set_log_level(loglvl)
    config.dataset_config = data_config.datasets[dataset_name]
    src, dst = get_paths(dataset_name, debug)
    ds = load_datasets(src)
    interp = interpolate_datasets(ds)
    save_datasets(interp, dst)

if __name__ == '__main__':
    p = argparse.ArgumentParser()
    p.add_argument('dataset_name', choices=['new_york'])
    p.add_argument('--debug', action='store_true')
    p.add_argument('--log_level', type=int, default=2)
    args = p.parse_args()

    dask.config.set(
        scheduler='single-threaded' if args.debug else 'processes'
    )
    main(args.dataset_name, args.debug, args.log_level)


In [ ]:
!python interpolator.py new_york --debug --log_level 3



In [ ]:
# 1. Zip the interpolated_debug directory
!zip -r /content/interpolated_debug.zip /content/data/new_york/interpolated_debug

# 2. Mount Google Drive (if not already mounted)
from google.colab import drive
drive.mount('/content/drive')

# 3. Copy the ZIP into your MyDrive
!cp /content/interpolated_debug.zip /content/drive/MyDrive/


In [ ]:


# 2. Load with pandas
import pandas as pd

# replace with your actual path
path = "/content/data/new_york/interpolated_debug/test.parquet/part.0.parquet"

# Read the whole file (or just the first N rows)
df = pd.read_parquet(path, engine="pyarrow")   # or engine="fastparquet"
print(df.shape)
df.head()   # show first 5 rows

# 3. Get a quick summary
df.info()
df.describe()


In [ ]:
%%writefile sliding_window.py
#!/usr/bin/env python3
"""
Sliding‑Window pipeline using Dask for large-scale AIS data.
Splits long tracks into overlapping (train/valid) or non‑overlapping (test) sub‑tracks.
"""
import os
import sys
import argparse

import numpy as np
import pandas as pd
import dask
import dask.dataframe as dd

import config
import data_config
from utils import clear_path

# ── Sliding‑window functions ── #

def window_track_long_term_train(track: pd.DataFrame, movement: int) -> pd.DataFrame:
    """Breaks a single track into overlapping windows for training/validation."""
    H = config.length_of_history
    F = config.length_into_the_future
    step = int(movement / config.interpolation_time_gap)
    n = len(track) - (H + F)
    idxs = [np.arange(i, i + H + F + 1) for i in range(0, n, step)]
    if not idxs:  # Handle empty list of indices
        return pd.DataFrame(columns=track.columns) # Return empty DataFrame with original columns
    return pd.concat(
        [track.iloc[ix].reset_index(drop=True) for ix in idxs],
        ignore_index=True
    )


def window_track_long_term_test(track: pd.DataFrame) -> pd.DataFrame:
    """Breaks a single track into non-overlapping windows for testing."""
    H = config.length_of_history
    F = config.length_into_the_future
    n = len(track) - (H + F)
    idxs = [np.arange(i, i + H + F + 1) for i in range(0, n, H)]
    if not idxs: # Handle empty list of indices
         return pd.DataFrame(columns=track.columns) # Return empty DataFrame with original columns
    return pd.concat(
        [track.iloc[ix].reset_index(drop=True) for ix in idxs],
        ignore_index=True
    )


def window_partition(df: pd.DataFrame, fn, *args) -> pd.DataFrame:
    """Apply sliding-window fn per track, and explicitly build the output DataFrame."""
    results = []
    # Iterate through groups and apply the windowing function
    for track_id, group_df in df.groupby('track'):
        windowed_df = fn(group_df, *args)
        if not windowed_df.empty:
            # Add 'track' as a column to the windowed DataFrame
            windowed_df['track'] = track_id
            results.append(windowed_df)

    if not results:
        # Return an empty DataFrame with expected columns if no windows were created
        # We need to infer the columns from the original DataFrame and the added 'track'
        # This might be tricky with Dask metadata, but for a partition, we can attempt
        # to get columns from the input or a sample. Let's use input columns + 'track'.
        cols = list(df.columns) + ['track']
        # Assuming the data types for the original columns are preserved,
        # and 'track' is the same dtype as the group key.
        # This is a simplified meta; might need refinement based on actual data types.
        # For now, let's return an empty DataFrame that *should* be compatible.
        # This might require more robust metadata handling for production use with Dask.
        # However, for debugging the 'level_1' error, let's see if this structure helps.
        return pd.DataFrame(columns=cols).set_index('track')


    # Concatenate the results from all tracks in this partition
    partition_result_df = pd.concat(results, ignore_index=True)

    # Set 'track' as the index
    partition_result_df = partition_result_df.set_index('track')

    # The index name might be None after set_index if the original 'track' column had no name.
    # Ensure the index is named 'track'
    partition_result_df.index.name = 'track'


    # Drop any residual 'level_0' or 'level_1' that might have been implicitly created
    # during the concatenation or index operations, though the manual construction
    # should ideally prevent 'level_1' conflicts from reset_index.
    partition_result_df = partition_result_df.drop(columns=['level_0', 'level_1'], errors='ignore')


    return partition_result_df


# ── I/O helpers ── #

def get_paths(dataset_name: str, debug: bool = False):
    """Determine source and destination directories for input/output."""
    base = os.path.join(config.data_directory, dataset_name)
    src_dbg = os.path.join(base, 'interpolated_debug')
    src_std = os.path.join(base, 'interpolated')
    if debug and os.path.isdir(src_dbg):
        src = src_dbg
    elif os.path.isdir(src_std):
        src = src_std
    elif os.path.isdir(src_dbg):
        src = src_dbg
    else:
        raise FileNotFoundError(f"Neither '{src_std}' nor '{src_dbg}' exists")
    dst = os.path.join(base, 'windowed_debug' if debug else 'windowed')
    os.makedirs(dst, exist_ok=True)
    return src, dst


def load_split(src: str, split: str) -> dd.DataFrame:
    """Load a split (train/valid/test) from parquet and ensure track index."""
    path = os.path.join(src, f"{split}.parquet")
    ddf = dd.read_parquet(path)
    if ddf.index.name != 'track':
        ddf = ddf.map_partitions(
            lambda df: df.set_index(pd.Index(df.index, name='track')),
            meta=ddf._meta
        )
    return ddf


def save_split(ddf: dd.DataFrame, dst: str, name: str):
    """Write a windowed split to parquet with clear logging."""
    out = os.path.join(dst, f"{name}.parquet")
    clear_path(out)
    ddf.to_parquet(out, schema='infer')
    print(f"→ Saved {name} to {out}")

# ── Main ── #

def main(dataset_name: str, debug: bool, loglvl: int):
    config.set_log_level(loglvl)
    ds_cfg = data_config.datasets[dataset_name]

    src, dst = get_paths(dataset_name, debug)
    splits = {
        'train': load_split(src, 'train'),
        'valid': load_split(src, 'valid'),
        'test':  load_split(src, 'test'),
    }

    outputs = {}
    # Overlapping windows for train & valid
    for split in ('train', 'valid'):
        outputs[f"{split}_long_term_train"] = splits[split].map_partitions(
            window_partition,
            window_track_long_term_train,
            ds_cfg.sliding_window_movement,
            meta=splits[split]._meta
        )
    # Non-overlapping windows for test & valid
    for split in ('test', 'valid'):
        outputs[f"{split}_long_term_test"] = splits[split].map_partitions(
            window_partition,
            window_track_long_term_test,
            meta=splits[split]._meta
        )

    for name, ddf in outputs.items():
        save_split(ddf, dst, name)

# Check if running in an interactive environment (like Colab)
def is_interactive():
    import __main__ as main
    return not hasattr(main, '__file__')

if __name__ == '__main__':
    # In interactive mode, directly call main with hardcoded values from the !python command
    main('new_york', debug=True, loglvl=3)

In [ ]:
import os
import dask.dataframe as dd
import pandas as pd # Import pandas to create metadata DataFrame
import shutil # Import shutil for removing directories
from utils import clear_path

# Import necessary functions from the sliding_window script
from sliding_window import (
    get_paths,
    load_split,
    window_partition,
    window_track_long_term_train,
    window_track_long_term_test,
    save_split
)

# Import config and data_config
import config
import data_config

# Hardcoded values from the previous execution attempt
dataset_name = 'new_york'
debug = True
loglvl = 3

# Set log level
config.set_log_level(loglvl)

# Get source and destination paths
src, dst = get_paths(dataset_name, debug)

# Load the interpolated data splits
splits = {
    'train': load_split(src, 'train'),
    'valid': load_split(src, 'valid'),
    'test':  load_split(src, 'test'),
}

outputs = {}
ds_cfg = data_config.datasets[dataset_name]

# --- Explicitly define metadata for map_partitions ---
# Get columns and dtypes from one of the input splits
# Assuming all splits have the same column structure
input_meta = splits['train']._meta

# Define the expected columns and dtypes after window_partition
# This should include the original columns (excluding the index if it was not a column)
# and potentially 'track' as the index.
# The original columns in the interpolated data were:
# 'base_datetime', 'mmsi', 'lat', 'lon', 'sog', 'cog', 'heading', 'status', 'year', 'month', 'track' (index)
# The window_partition function is expected to return a DataFrame with 'track' as index
# and these columns as data columns.
# We need to create a pandas DataFrame metadata object that reflects this structure.

# Get data columns from input meta (excluding the index)
data_cols_meta = input_meta.reset_index().drop(columns=['track', 'level_0', 'level_1'], errors='ignore')

# Create a new metadata DataFrame with 'track' as the index and the data columns
# The dtype of 'track' should match the original index dtype
track_dtype = input_meta.index.dtype

# Create an empty pandas DataFrame with the desired column structure and dtypes
# The index will be named 'track' with the correct dtype
# The columns will be the data columns with their original dtypes
expected_columns = data_cols_meta.columns
expected_dtypes = data_cols_meta.dtypes

# Create an empty DataFrame with the correct structure
output_meta_dict = {col: pd.Series(dtype=dtype) for col, dtype in zip(expected_columns, expected_dtypes)}
output_meta = pd.DataFrame(output_meta_dict)
output_meta.index = pd.Index([], dtype=track_dtype, name='track')

# Ensure 'base_datetime' is treated as object/int64 before conversion back to datetime if needed later
if 'base_datetime' in output_meta.columns:
    output_meta['base_datetime'] = output_meta['base_datetime'].astype('int64')


# Apply overlapping windows for train & valid
for split in ('train', 'valid'):
    outputs[f"{split}_long_term_train"] = splits[split].map_partitions(
        window_partition,
        window_track_long_term_train,
        ds_cfg.sliding_window_movement,
        meta=output_meta # Pass explicit metadata
    )

# Apply non-overlapping windows for test & valid
for split in ('test', 'valid'):
    outputs[f"{split}_long_term_test"] = splits[split].map_partitions(
        window_partition,
        window_track_long_term_test,
        meta=output_meta # Pass explicit metadata
    )

# Save the windowed datasets
# Modified save_split function with checks
def save_split_with_checks(ddf: dd.DataFrame, dst: str, name: str):
    """Write a windowed split to parquet with clear logging and checks."""
    out = os.path.join(dst, f"{name}.parquet")
    clear_path(out)
    print(f"Attempting to save {name} to {out}")
    try:
        ddf.to_parquet(out, schema='infer')
        print(f"Save operation for {out} completed.")

        # Add checks after saving
        print(f"Checking saved location: {out}")
        if not os.path.exists(out):
            print(f"Error: Output directory {out} was not created after saving.")
            raise FileNotFoundError(f"Output directory {out} was not created after saving.")
        if not os.path.isdir(out):
             print(f"Error: Output path {out} is not a directory after saving.")
             raise NotADirectoryError(f"Output path {out} is not a directory after saving.")

        output_dir_contents = os.listdir(out)
        print(f"Contents of output directory {out}: {output_dir_contents}")

        parquet_files_found = any(f.endswith('.parquet') for f in output_dir_contents)
        if not parquet_files_found:
             print(f"Error: Output directory {out} is empty or does not contain .parquet files after saving.")
             raise FileNotFoundError(f"Output directory {out} is empty or does not contain .parquet files after saving.")
        print(f"Found .parquet files in output directory {out}.")

        print(f"→ Saved {name} successfully to {out}")

    except Exception as e:
        print(f"Error during save or check of output location {out}: {e}")
        # Clean up the incomplete output directory
        if os.path.exists(out):
             print(f"Cleaning up incomplete output directory: {out}")
             if os.path.isdir(out):
                 shutil.rmtree(out)
             else:
                 os.remove(out)
        raise # Re-raise the exception


for name, ddf in outputs.items():
    save_split_with_checks(ddf, dst, name)

print("Sliding window processing complete.")

In [ ]:


# 2. Load with pandas
import pandas as pd

# replace with your actual path
path = "/content/data/new_york/windowed_debug/test_long_term_test.parquet"

# Read the whole file (or just the first N rows)
df = pd.read_parquet(path, engine="pyarrow")   # or engine="fastparquet"
print(df.shape)
df.head()   # show first 5 rows

# 3. Get a quick summary
df.info()
df.describe()


In [ ]:
# 1. Zip the windowed_debug directory
!zip -r /content/windowed_debug.zip /content/data/new_york/windowed_debug

# 2. Mount Google Drive (if not already mounted)
from google.colab import drive
drive.mount('/content/drive')

# 3. Copy the ZIP into your MyDrive
!cp /content/windowed_debug.zip /content/drive/MyDrive/


In [ ]:
!pip install --upgrade dask-ml scikit-learn dask pandas pyarrow fastparquet

In [ ]:
%%writefile formatter.py
# formatter.py
import argparse
import logging
import os
import sys
import gc
import shutil
from typing import Optional, Any
import hashlib
import numpy as np
import pandas as pd
import dask
import dask.dataframe as dd
import json # Added for saving all_categories

# Assuming config and utils are in the same directory or accessible via PYTHONPATH
import config
from data_config import datasets
import utils

# Initialize OneHotEncoder type hint to Any if dask_ml is not available
OneHotEncoderType = Any
try:
    from dask_ml.preprocessing import OneHotEncoder
    use_ml = True
    OneHotEncoderType = OneHotEncoder
except ImportError:
    use_ml = False
    logging.warning("dask-ml not found. Falling back to dd.get_dummies, which may lead to "
                    "inconsistent feature sets if categories vary across data splits. "
                    "It is highly recommended to install dask-ml for robust one-hot encoding.")


engine = None
try:
    import pyarrow  # noqa: F401
    engine = 'pyarrow'
except ImportError:
    try:
        import fastparquet  # noqa: F401
        engine = 'fastparquet'
    except ImportError:
        sys.exit("Missing dependency 'pyarrow' or 'fastparquet'.")


def setup_logging(save_log: bool, level: int):
    fmt = '%(asctime)s %(levelname)s %(message)s'
    if save_log:
        logging.basicConfig(filename='format.log', level=level, format=fmt)
    else:
        logging.basicConfig(level=level, format=fmt)


def repartition_if_needed(df: dd.DataFrame, name: str, base: str) -> dd.DataFrame:
    tmp = os.path.join(base, f'.tmp_{name}.parquet')
    path = os.path.join(base, f'{name}.parquet')

    if os.path.exists(tmp):
        shutil.rmtree(tmp) if os.path.isdir(tmp) else os.remove(tmp)

    original_ddf = dd.read_parquet(path, engine=engine)
    df = original_ddf.repartition(partition_size='4000MB', force=True)
    os.makedirs(os.path.dirname(tmp), exist_ok=True)
    dd.to_parquet(df, tmp, engine=engine, schema='infer', write_metadata_file=True)
    return dd.read_parquet(tmp, engine=engine)


def load_windowed(base: str, names: list, engine: str) -> dict:
    out = {}
    for n in names:
        original_path = os.path.join(base, f'{n}.parquet')
        raw = dd.read_parquet(original_path, engine=engine)
        out[n] = raw
    return out


def one_hot_encode(df: dd.DataFrame, categorical_columns: list,
                   fitted_encoder: Optional[OneHotEncoderType] = None,
                   all_known_categories: Optional[dict] = None) -> (dd.DataFrame, Optional[OneHotEncoderType]):
    cols_to_encode = [c for c in categorical_columns if c in df.columns]

    if not cols_to_encode:
        return df, fitted_encoder

    if all_known_categories:
        logging.info(f"Applying global known categories to {cols_to_encode} for this DataFrame.")
        for col in cols_to_encode:
            if col in all_known_categories:
                df[col] = df[col].astype(pd.CategoricalDtype(categories=all_known_categories[col]))
            else:
                logging.warning(f"Categorical column '{col}' missing from 'all_known_categories'. "
                                "Categories will be inferred locally for this column.")
                df[col] = df[col].astype('category') # Fallback to infer locally
    else:
        logging.warning("No global known categories provided. Inferring categories locally. "
                        "This may lead to inconsistent features if categories differ across splits.")
        df = df.categorize(columns=cols_to_encode)

    if use_ml:
        if fitted_encoder is None:
            logging.info(f"Fitting OneHotEncoder on categorical columns: {cols_to_encode}")
            fitted_encoder = OneHotEncoder(sparse_output=False)
            fitted_encoder.fit(df[cols_to_encode])

        transformed_df = fitted_encoder.transform(df[cols_to_encode])

        df = df.drop(cols_to_encode, axis=1)
        df = dd.concat([df, transformed_df], axis=1)

    else: # Fallback when dask_ml is not available, using dd.get_dummies
        logging.warning("Using dd.get_dummies without a pre-fitted encoder. "
                        "Feature consistency relies on all_known_categories being pre-applied and fillna(0).")
        df = dd.get_dummies(df, columns=cols_to_encode, dtype='bool')

    df = df.map_partitions(lambda pdf: pdf.apply(pd.to_numeric, errors='coerce').astype('float32'))
    return df, fitted_encoder


def reshape_partition(part: pd.DataFrame, ts: int) -> np.ndarray:
    part = part.replace('undefined', np.nan)
    part = part.apply(pd.to_numeric, errors='coerce').astype('float32')

    if part.empty:
        return np.empty((0, ts, part.shape[1]), dtype='float32')

    arr = part.to_numpy()
    num_windows = len(arr) // ts
    if num_windows == 0:
        return np.empty((0, ts, part.shape[1]), dtype='float32')

    rows = [np.arange(i * ts, (i + 1) * ts) for i in range(num_windows)]

    return np.stack([arr[r] for r in rows])


def format_and_save(dfs: dict, out: str, conserve: bool = True, engine: str = None, feature_cols_map: dict = None):
    utils.clear_path(out)
    os.makedirs(out, exist_ok=True)

    if not dfs or all(ddf.npartitions == 0 for ddf in dfs.values()):
        # Ensure features.csv is still created, even if empty, for DataLoader to not fail
        pd.DataFrame(columns=['FeatureName', 'dtype', 'IsTarget']).to_csv(os.path.join(out, 'features.csv'), index=False)
        logging.warning("No dataframes or all dataframes are empty. Exiting format_and_save.")
        return

    if feature_cols_map is None:
        logging.error("feature_cols_map is None. Cannot ensure consistent columns for saving NumPy arrays. Exiting.")
        pd.DataFrame(columns=['FeatureName', 'dtype', 'IsTarget']).to_csv(os.path.join(out, 'features.csv'), index=False)
        return

    # Determine latitude and longitude indices (using 'lat' and 'lon' as per data)
    lat_idx = feature_cols_map.get('lat')
    lon_idx = feature_cols_map.get('lon')

    if lat_idx is None or lon_idx is None:
        logging.error("Latitude or Longitude column (named 'lat' or 'lon') not found in feature_cols_map. Cannot generate Y data correctly. Exiting.")
        pd.DataFrame(columns=['FeatureName', 'dtype', 'IsTarget']).to_csv(os.path.join(out, 'features.csv'), index=False)
        return

    # Define the indices for the Y data (latitude, longitude)
    y_feature_indices = [lat_idx, lon_idx] # This maintains the order [lat, lon]

    print(f"\n--- DEBUG: Y Feature Indices for Target ---")
    print(f"  'lat' found at index: {lat_idx}")
    print(f"  'lon' found at index: {lon_idx}")
    print(f"  Y data will be sliced using indices: {y_feature_indices}")
    print(f"  Corresponding feature names: {[list(feature_cols_map.keys())[i] for i in y_feature_indices]}")
    print(f"-------------------------------------------\n")


    # Store feature names for X and Y in features.csv
    feats_df_data = []
    for col_name, idx in feature_cols_map.items():
        is_target = col_name in ['lat', 'lon']
        feats_df_data.append({'FeatureName': col_name, 'dtype': 'float32', 'IsTarget': is_target})

    feats_df = pd.DataFrame(feats_df_data)
    # Sort by the original order to ensure consistent indexing in the CSV
    feats_df['Order'] = feats_df['FeatureName'].map(feature_cols_map)
    feats_df = feats_df.sort_values(by='Order').drop(columns=['Order'])

    feats_df.to_csv(os.path.join(out, 'features.csv'), index=False)
    logging.info(f"Saved features metadata to {os.path.join(out, 'features.csv')}")

    tsf = config.length_of_history + config.length_into_the_future + 1

    for nm, ddf in dfs.items():
        sd = os.path.join(out, nm)
        os.makedirs(sd, exist_ok=True)
        logging.info(f"Processing dataset: {nm}")

        if ddf.npartitions == 0:
            logging.warning(f"Dataset {nm} has no partitions. Creating empty directories.")
            for gap in config.time_gaps:
                mins = int(gap / 60)
                os.makedirs(os.path.join(sd, f'{mins}_min_time_gap_x'), exist_ok=True)
                os.makedirs(os.path.join(sd, f'{mins}_min_time_gap_y'), exist_ok=True)
            continue

        logging.info(f"Aligning columns for {nm} to master feature set.")
        aligned_ddf = ddf[list(feature_cols_map.keys())].fillna(0.0)

        for gap in config.time_gaps:
            step = int(gap / config.interpolation_time_gap)
            xi = np.arange(0, config.length_of_history)
            yi = np.arange(config.length_of_history, tsf, step)
            xi = xi[xi < tsf]; yi = yi[yi < tsf]

            mins = int(gap / 60)
            xdir = os.path.join(sd, f'{mins}_min_time_gap_x'); os.makedirs(xdir, exist_ok=True)
            ydir = os.path.join(sd, f'{mins}_min_time_gap_y'); os.makedirs(ydir, exist_ok=True)
            logging.info(f"    Processing time gap {mins} min for {nm}")

            if conserve:
                delayed_saves = []
                for i in range(aligned_ddf.npartitions):
                    pdf = aligned_ddf.partitions[i].compute()
                    arr = reshape_partition(pdf, tsf)

                    x_arr = arr[:, xi, :] # Keep all features for X
                    # Corrected for IndexError: shape mismatch
                    y_arr = arr[:, yi][:, :, y_feature_indices] # Select lat/lon for Y

                    xp, yp = os.path.join(xdir, f'{i}.npy'), os.path.join(ydir, f'{i}.npy')

                    def save_arrays(xa, ya, xp, yp):
                        np.save(xp, xa); np.save(yp, ya)
                        return xa.shape[0], ya.shape[0]

                    delayed_saves.append(dask.delayed(save_arrays)(x_arr, y_arr, xp, yp))

                dask.compute(*delayed_saves)
                logging.info(f"    Saved {len(delayed_saves)} partitions for {nm} and {mins} min gap (conserve mode).")
            else:
                full = aligned_ddf.compute()
                arr = reshape_partition(full, tsf)
                x_arr = arr[:, xi, :] # Keep all features for X
                # Corrected for IndexError: shape mismatch
                y_arr = arr[:, yi][:, :, y_feature_indices] # Select lat/lon for Y

                np.save(os.path.join(sd, f'{mins}_min_time_gap_x.npy'), x_arr)
                np.save(os.path.join(sd, f'{mins}_min_time_gap_y.npy'), y_arr)
                logging.info(f"    Saved full arrays for {nm} and {mins} min gap (non-conserve mode).")
                del full; gc.collect()

    print("Formatting process complete.")


if __name__=='__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('dataset_name', choices=datasets.keys())
    parser.add_argument('-l','--log_level', type=int, default=2, choices=[0,1,2,3,4])
    parser.add_argument('-s','--save_log', action='store_true')
    parser.add_argument('--debug', action='store_true')

    args, unknown = parser.parse_known_args()

    if unknown:
        logging.warning(f"Found unknown arguments: {unknown}. These will be ignored.")
        logging.warning("If these include a kernel JSON path, ensure you're calling the script correctly.")


    config.set_log_level(args.log_level)
    setup_logging(args.save_log, args.log_level)

    base = f"/content/data/{args.dataset_name}"
    wdir = os.path.join(base, 'windowed' + ('_debug' if args.debug else ''))
    odir = os.path.join(base, 'formatted' + ('_debug' if args.debug else ''))
    names = [
        'train_long_term_train',
        'test_long_term_test',
        'valid_long_term_train',
        'valid_long_term_test'
    ]

    dfs = load_windowed(wdir, names, engine)

    fitted_ohe = None
    feature_columns_after_ohe = None

    # --- Step 1: Collect ALL unique categories from ALL data splits ---
    all_categories = {}
    logging.info("Collecting global unique categories for categorical columns across all datasets.")
    for col in config.categorical_columns:
        if any(col in ddf.columns for ddf in dfs.values()):
            combined_series_list = [ddf[col] for ddf in dfs.values() if col in ddf.columns]

            combined_series = dd.concat(combined_series_list)

            unique_cats = combined_series.astype('category').cat.as_known().cat.categories

            all_categories[col] = list(unique_cats)
    logging.info(f"Collected global categories: {all_categories}")

    # 2. Process training data first to fit the OneHotEncoder
    train_name = 'train_long_term_train'
    if train_name in dfs:
        logging.info(f"Processing training data ({train_name}) to fit OneHotEncoder.")
        dfs[train_name], fitted_ohe = one_hot_encode(
            dfs[train_name], config.categorical_columns, fitted_ohe,
            all_known_categories=all_categories
        )
        feature_columns_after_ohe = dfs[train_name].columns.tolist()
        logging.info(f"Training data columns after OHE: {feature_columns_after_ohe}")
    else:
        logging.error(f"Training data '{train_name}' not found. Cannot fit OneHotEncoder.")
        sys.exit(1)


    # 3. Process all other datasets using the fitted encoder
    for n in names:
        if n == train_name:
            continue
        if n in dfs:
            logging.info(f"Processing data split: {n} (transforming using fitted encoder).")
            dfs[n], _ = one_hot_encode(
                dfs[n], config.categorical_columns, fitted_ohe,
                all_known_categories=all_categories
            )
            logging.info(f"    Columns for {n} after OHE: {dfs[n].columns.tolist()}")
        else:
            logging.warning(f"Data split '{n}' not found. Skipping.")


    # 4. Pass the master feature columns to format_and_save
    if feature_columns_after_ohe is None:
        logging.error("No master feature columns defined after processing training data. Exiting.")
        sys.exit(1)

    master_feature_cols_map = {col: i for i, col in enumerate(feature_columns_after_ohe)}

    format_and_save(dfs, odir, conserve=True, engine=engine, feature_cols_map=master_feature_cols_map)

    # --- MOVED: Save all_categories for consistent one-hot encoding during prediction ---
    all_categories_path = os.path.join(odir, 'all_categories.json')
    with open(all_categories_path, 'w') as f:
        json.dump(all_categories, f, indent=4)
    logging.info(f"Saved all_categories to {all_categories_path}")
    # --- END MOVED SECTION ---

    logging.info("Formatter script finished successfully.")


In [ ]:
!python formatter.py new_york --debug -l 3 -s

In [ ]:
import numpy as np

# adjust this path to wherever your formatted files live
x = np.load('/content/data/new_york/formatted_debug/test_long_term_test/30_min_time_gap_x/0.npy')
y = np.load('/content/data/new_york/formatted_debug/test_long_term_test/30_min_time_gap_y/0.npy')

print("X shape:", x.shape)
print("Y shape:", y.shape)


In [ ]:
# 2. Define source folder
src = "/content/data/new_york/formatted_debug"

# 3. Define output .zip path in MyDrive
zip_path = "/content/drive/MyDrive/formatted_debug.zip"

# 4. Create the zip
print(f"Zipping {src} → {zip_path}")
shutil.make_archive(base_name=zip_path.replace('.zip',''),
                    format='zip',
                    root_dir=src)

print("Done! 🎉 You’ll find formatted_debug.zip in the root of your MyDrive.")

In [2]:
from google.colab import drive
drive.mount('/content/drive')  # grant Colab access to your Drive

# then copy the file into the working directory
!cp "/content/drive/MyDrive/formatted_debug.zip" .


Mounted at /content/drive


In [ ]:
!rm -rf /content/data/new_york/formatted_debug

In [3]:
!unzip formatted_debug.zip -d formatted_debug



Archive:  formatted_debug.zip
   creating: formatted_debug/test_long_term_test/
   creating: formatted_debug/train_long_term_train/
   creating: formatted_debug/valid_long_term_test/
   creating: formatted_debug/valid_long_term_train/
  inflating: formatted_debug/features.csv  
  inflating: formatted_debug/all_categories.json  
   creating: formatted_debug/test_long_term_test/30_min_time_gap_x/
   creating: formatted_debug/test_long_term_test/30_min_time_gap_y/
  inflating: formatted_debug/test_long_term_test/30_min_time_gap_x/53.npy  
  inflating: formatted_debug/test_long_term_test/30_min_time_gap_x/62.npy  
  inflating: formatted_debug/test_long_term_test/30_min_time_gap_x/50.npy  
  inflating: formatted_debug/test_long_term_test/30_min_time_gap_x/88.npy  
  inflating: formatted_debug/test_long_term_test/30_min_time_gap_x/68.npy  
  inflating: formatted_debug/test_long_term_test/30_min_time_gap_x/22.npy  
  inflating: formatted_debug/test_long_term_test/30_min_time_gap_x/99.npy  
  

In [ ]:
import numpy as np

# Path to your .npy file
path = "/content/data/new_york/formatted_debug/test_long_term_test/30_min_time_gap_x/0.npy"

# Load the array
arr = np.load(path, allow_pickle=True)   # set allow_pickle=True only if you know it contains pickled objects

# Inspect shape and dtype
print("Array shape:", arr.shape)
print("Array dtype:", arr.dtype)

# Peek at the first few entries
print("First 5 elements:\n", arr[:5])


In [ ]:
!du -sh /content/data/new_york/formatted_debug

In [ ]:
import pandas as pd

# Load your feature definitions
features = pd.read_csv('/content/data/new_york/formatted_debug/features.csv')
features.head(13)




In [ ]:
%%writefile loading_utils.py
import random
import os
import shutil

import numpy as np
import tensorflow as tf

def set_seed(seed):
    """
    Set relevant random seeds

    :param seed:
    :return:
    """
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)


def clear_path(path):
    """
    Delete any files or directories from a path

    :param path: Path to remove
    :type path: str
    :return: None
    """
    if os.path.exists(path):
        if os.path.isfile(path):
            os.remove(path)
        else:
            shutil.rmtree(path)

def total_system_ram():
    """
    Get total system ram, in bytes

    :return:
    """
    return os.sysconf('SC_PAGE_SIZE') * os.sysconf('SC_PHYS_PAGES')

In [ ]:
!python loading_utils.py

In [11]:
%%writefile normalizer.py
import pandas as pd
import numpy as np
from typing import Dict, Any, List, Optional

class Normalizer:
    """
    Handles data normalization (e.g., Min-Max scaling).
    Can calculate factors from a DataFrame and apply normalization to new data.
    """
    def __init__(self, method: str = 'min_max'):
        if method not in ['min_max']: # Can extend to 'z_score' if needed
            raise ValueError(f"Normalization method '{method}' not supported.")
        self.method = method
        self.normalization_factors: Optional[Dict[str, Dict[str, float]]] = None

    def get_normalization_factors(self, df: pd.DataFrame, features: List[str]) -> Dict[str, Dict[str, float]]:
        """
        Calculates normalization factors (min/max) for specified features from a DataFrame.
        This method is typically called only on the training data.

        Args:
            df (pd.DataFrame): The DataFrame to calculate factors from.
            features (List[str]): List of column names (features) to calculate factors for.

        Returns:
            Dict[str, Dict[str, float]]: A dictionary where keys are feature names and values
                                         are dictionaries containing 'min' and 'max' values.
        """
        factors = {}
        for feature in features:
            if feature in df.columns and pd.api.types.is_numeric_dtype(df[feature]):
                min_val = df[feature].min()
                max_val = df[feature].max()
                if max_val == min_val: # Avoid division by zero
                    min_val = min_val - 1e-6
                    max_val = max_val + 1e-6
                factors[feature] = {'min': float(min_val), 'max': float(max_val)}
            else:
                print(f"Warning: Feature '{feature}' not found or not numeric in DataFrame for factor calculation.")
        self.normalization_factors = factors
        return factors

    def normalize_data(self, data: np.ndarray, normalization_factors: Dict[str, Dict[str, float]],
                       feature_mapping: Dict[str, int]) -> np.ndarray:
        """
        Applies normalization to a numpy array using provided normalization factors.
        Assumes `data` is 3D (batch, sequence_length, features).

        Args:
            data (np.ndarray): The numpy array to normalize.
            normalization_factors (Dict[str, Dict[str, float]]): Factors to use for normalization.
            feature_mapping (Dict[str, int]): A dictionary mapping feature names to their
                                              index in the last dimension of the numpy array.

        Returns:
            np.ndarray: The normalized numpy array.
        """
        if not normalization_factors:
            print("Warning: No normalization factors provided. Returning data unnormalized.")
            return data

        normalized_data = data.copy()

        for feature_name, factors in normalization_factors.items():
            if feature_name in feature_mapping:
                col_idx = feature_mapping[feature_name]
                min_val = factors['min']
                max_val = factors['max']

                if self.method == 'min_max':
                    # Apply Min-Max scaling: normalized = (original - min) / (max - min)
                    # Handle the case where max_val == min_val to prevent division by zero
                    if (max_val - min_val) != 0:
                        normalized_data[:, :, col_idx] = (normalized_data[:, :, col_idx] - min_val) / (max_val - min_val)
                    else:
                        # If min == max, set to 0.5 (midpoint) or 0 if a specific behavior is desired
                        normalized_data[:, :, col_idx] = 0.5 # or 0.0 depending on convention
            else:
                # This could happen if a factor exists for a feature not present in the current 'data' array
                print(f"Warning: Feature '{feature_name}' from normalization factors not found in provided data mapping. Skipping.")
        return normalized_data

    def unnormalize_data(self, data: np.ndarray, normalization_factors: Dict[str, Dict[str, float]],
                         feature_mapping: Dict[str, int]) -> np.ndarray:
        """
        Applies inverse normalization (un-normalizes) to a numpy array.
        Assumes `data` is 3D (batch, sequence_length, features).

        Args:
            data (np.ndarray): The numpy array to unnormalize.
            normalization_factors (Dict[str, Dict[str, float]]): Factors to use for unnormalization.
            feature_mapping (Dict[str, int]): A dictionary mapping feature names to their
                                              index in the last dimension of the numpy array.

        Returns:
            np.ndarray: The unnormalized numpy array.
        """
        if not normalization_factors:
            print("Warning: No normalization factors provided. Returning data as is.")
            return data

        unnormalized_data = data.copy()

        for feature_name, factors in normalization_factors.items():
            if feature_name in feature_mapping:
                col_idx = feature_mapping[feature_name]
                min_val = factors['min']
                max_val = factors['max']

                if self.method == 'min_max':
                    # Apply inverse Min-Max scaling: original = normalized * (max - min) + min
                    unnormalized_data[:, :, col_idx] = unnormalized_data[:, :, col_idx] * (max_val - min_val) + min_val
            else:
                # This could happen if a factor exists for a feature not present in the current 'data' array
                pass
        return unnormalized_data

# Self-test for Normalizer
if __name__ == "__main__":
    print("Running self-test for normalizer.py...")

    # Create dummy data
    data = np.random.rand(10, 5, 3) # 10 samples, 5 timesteps, 3 features

    # Simulate features.csv column names and their mapping to indices
    # This is what formatter.py's features.csv would tell us.
    feature_names = ['feature_A', 'feature_B', 'feature_C']
    feature_mapping = {name: idx for idx, name in enumerate(feature_names)}

    # Create a DataFrame for factor calculation (only training data should be used for this)
    # Let's assume a simplified 2D DataFrame for this, representing the 'raw' features before windowing
    df_for_factors = pd.DataFrame({
        'feature_A': np.random.uniform(10, 20, 100),
        'feature_B': np.random.uniform(0, 1, 100),
        'feature_C': np.random.uniform(-5, 5, 100),
        'categorical_feature': ['X', 'Y', 'Z'] * 33 + ['X']
    })

    normalizer = Normalizer(method='min_max')

    # Test factor calculation
    print("\nTesting factor calculation...")
    numeric_features_for_factors = ['feature_A', 'feature_B', 'feature_C']
    factors = normalizer.get_normalization_factors(df_for_factors, numeric_features_for_factors)
    print("Calculated Factors:", factors)

    assert 'feature_A' in factors and 'feature_B' in factors and 'feature_C' in factors
    assert factors['feature_A']['min'] >= 10 and factors['feature_A']['max'] <= 20
    assert factors['feature_B']['min'] >= 0 and factors['feature_B']['max'] <= 1
    assert factors['feature_C']['min'] >= -5 and factors['feature_C']['max'] <= 5

    # Test normalization
    print("\nTesting normalization...")
    original_data = np.array([
        [[15.0, 0.5, 0.0], [12.0, 0.2, -3.0]], # Sample 1
        [[18.0, 0.8, 4.0], [10.0, 0.0, -5.0]]  # Sample 2
    ], dtype=np.float32)

    # Make a 3D array with matching feature columns for test
    # This simulates the X array from formatter.py, with known values for `feature_A`, `feature_B`, `feature_C`
    test_X_data = np.zeros((2, 2, len(feature_names)), dtype=np.float32)
    test_X_data[0,0, feature_mapping['feature_A']] = 15.0
    test_X_data[0,0, feature_mapping['feature_B']] = 0.5
    test_X_data[0,0, feature_mapping['feature_C']] = 0.0
    test_X_data[0,1, feature_mapping['feature_A']] = 12.0
    test_X_data[0,1, feature_mapping['feature_B']] = 0.2
    test_X_data[0,1, feature_mapping['feature_C']] = -3.0

    test_X_data[1,0, feature_mapping['feature_A']] = 18.0
    test_X_data[1,0, feature_mapping['feature_B']] = 0.8
    test_X_data[1,0, feature_mapping['feature_C']] = 4.0
    test_X_data[1,1, feature_mapping['feature_A']] = 10.0
    test_X_data[1,1, feature_mapping['feature_B']] = 0.0
    test_X_data[1,1, feature_mapping['feature_C']] = -5.0

    # Example factors to use for normalization
    example_factors = {
        'feature_A': {'min': 10.0, 'max': 20.0},
        'feature_B': {'min': 0.0, 'max': 1.0},
        'feature_C': {'min': -5.0, 'max': 5.0},
    }

    normalized_data = normalizer.normalize_data(test_X_data, example_factors, feature_mapping)
    print("Normalized Data (first 2 samples, 2 timesteps):")
    print(normalized_data[:, :2, :]) # Print first 2 timesteps and all features

    # Verify values are between 0 and 1
    assert np.all((normalized_data >= -1e-6) & (normalized_data <= 1 + 1e-6))

    # Test unnormalization
    print("\nTesting unnormalization...")
    unnormalized_data = normalizer.unnormalize_data(normalized_data, example_factors, feature_mapping)
    print("Unnormalized Data (first 2 samples, 2 timesteps):")
    print(unnormalized_data[:, :2, :])

    # Verify unnormalized data is close to original_data
    assert np.allclose(unnormalized_data[:, :, feature_mapping['feature_A']], test_X_data[:, :, feature_mapping['feature_A']], atol=1e-5)
    assert np.allclose(unnormalized_data[:, :, feature_mapping['feature_B']], test_X_data[:, :, feature_mapping['feature_B']], atol=1e-5)
    assert np.allclose(unnormalized_data[:, :, feature_mapping['feature_C']], test_X_data[:, :, feature_mapping['feature_C']], atol=1e-5)

    print("\nNormalizer self-test PASSED!")

Writing normalizer.py


In [ ]:
!python normalizer.py

In [10]:
%%writefile loading.py
import pandas as pd
import numpy as np
from typing import Dict, Any

def read_ts_data(path: str) -> pd.DataFrame:
    """
    Reads time series data from a parquet file.
    Placeholder function. In a real scenario, this might handle errors,
    or convert specific column types.
    """
    print(f"Reading data from {path}")
    try:
        return pd.read_parquet(path)
    except Exception as e:
        print(f"Error reading parquet from {path}: {e}")
        return pd.DataFrame()

# Helper function for Haversine distance
def _haversine_distance(lat1, lon1, lat2, lon2):
    """
    Calculate the Haversine distance between two points on Earth (specified in decimal degrees).
    Returns distance in meters.
    """
    R = 6371000  # Radius of Earth in meters
    phi1, phi2 = np.radians(lat1), np.radians(lat2)
    delta_phi = np.radians(lat2 - lat1)
    delta_lambda = np.radians(lon2 - lon1)

    a = np.sin(delta_phi / 2)**2 + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    distance = R * c
    return distance

# Helper function to calculate initial bearing
def _calculate_initial_bearing(lat1, lon1, lat2, lon2):
    """
    Calculate the initial bearing from point 1 to point 2.
    Returns bearing in degrees (0-360).
    """
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])

    delta_lon = lon2 - lon1

    x = np.sin(delta_lon) * np.cos(lat2)
    y = np.cos(lat1) * np.sin(lat2) - (np.sin(lat1) * np.cos(lat2) * np.cos(delta_lon))

    initial_bearing = np.arctan2(x, y)

    # Convert to degrees and normalize to 0-360
    initial_bearing = np.degrees(initial_bearing)
    initial_bearing = (initial_bearing + 360) % 360

    return initial_bearing

def apply_transformations(df: pd.DataFrame, latitude_col: str = 'latitude',
                          longitude_col: str = 'longitude', timestamp_col: str = 'timestamp',
                          vessel_id_col: str = 'vessel_id') -> pd.DataFrame:
    """
    Applies initial feature engineering transformations to the DataFrame for vessel trajectories.
    These transformations should *not* include normalization or one-hot encoding,
    as those are handled by formatter.py and dataloader.py respectively.

    Args:
        df (pd.DataFrame): The input DataFrame. Expected to have 'latitude', 'longitude', 'timestamp', and 'vessel_id' columns.
        latitude_col (str): Name of the latitude column.
        longitude_col (str): Name of the longitude column.
        timestamp_col (str): Name of the timestamp column.
        vessel_id_col (str): Name of the vessel ID column.

    Returns:
        pd.DataFrame: The DataFrame with engineered features.
    """
    print("Applying initial feature engineering transformations...")

    # Ensure timestamp is datetime
    if timestamp_col in df.columns:
        df[timestamp_col] = pd.to_datetime(df[timestamp_col], errors='coerce')
        df = df.dropna(subset=[timestamp_col]) # Drop rows with invalid timestamps
        df = df.sort_values(by=[vessel_id_col, timestamp_col]).reset_index(drop=True)
    else:
        print(f"Warning: '{timestamp_col}' column not found. Cannot create time-based features or order data.")
        return df # Or raise error depending on strictness

    # Handle vessel-specific calculations
    if vessel_id_col in df.columns:
        # Group by vessel_id for calculations that should not cross vessel boundaries
        grouped = df.groupby(vessel_id_col)

        # Calculate time difference (in seconds)
        df['time_diff_sec'] = grouped[timestamp_col].diff().dt.total_seconds().fillna(0)

        # Calculate previous lat/lon for distance and bearing
        df['prev_latitude'] = grouped[latitude_col].shift(1)
        df['prev_longitude'] = grouped[longitude_col].shift(1)

        # Calculate segment distance (meters)
        # Use .apply(lambda x: ...) with a Series for performance on grouped data, if possible
        # Otherwise, fillna with values that result in 0 distance for the first point
        df['segment_distance_m'] = _haversine_distance(
            df['prev_latitude'].fillna(df[latitude_col]), # Use current lat for first point (distance 0)
            df['prev_longitude'].fillna(df[longitude_col]), # Use current lon for first point (distance 0)
            df[latitude_col],
            df[longitude_col]
        )

        # Calculate speed (meters/second and knots)
        df['speed_mps'] = df['segment_distance_m'] / df['time_diff_sec']
        # Handle division by zero for speed if time_diff_sec is 0
        df['speed_mps'] = df['speed_mps'].replace([np.inf, -np.inf], np.nan).fillna(0) # Fill nan after inf/nan conversion

        # 1 knot = 0.514444 meters/second
        df['speed_knots'] = df['speed_mps'] / 0.514444
        df['speed_knots'] = df['speed_knots'].fillna(0)

        # Calculate course/bearing (degrees)
        # For first point of a track, bearing is NaN, fill with 0 or last known.
        df['course_deg'] = _calculate_initial_bearing(
            df['prev_latitude'].fillna(df[latitude_col]), # Use current lat for first point
            df['prev_longitude'].fillna(df[longitude_col]), # Use current lon for first point
            df[latitude_col],
            df[longitude_col]
        )
        # The bearing for the first point will be NaN or inaccurate, fill with 0 or a sensible default
        df['course_deg'] = df['course_deg'].fillna(0)


        # Calculate change in speed and course (derivative features)
        df['speed_change_knots'] = grouped['speed_knots'].diff().fillna(0)
        # For course change, handle wrap-around (e.g., 350 to 10 deg is +20, not -340)
        df['course_change_deg'] = df['course_deg'] - grouped['course_deg'].shift(1)
        df['course_change_deg'] = df['course_change_deg'].apply(lambda x: (x + 180) % 360 - 180 if pd.notna(x) else 0) # Wrap-around
        df['course_change_deg'] = df['course_change_deg'].fillna(0)


        # Drop intermediate columns used for calculation
        df = df.drop(columns=['prev_latitude', 'prev_longitude'], errors='ignore')

    else:
        print(f"Warning: '{vessel_id_col}' column not found. Calculations for speed, distance, course will be global, not per-vessel.")
        # If no vessel_id_col, you'd apply global diff/shift which is usually incorrect for trajectories.
        # For a real pipeline, vessel_id_col would be mandatory.
        df['time_diff_sec'] = df[timestamp_col].diff().dt.total_seconds().fillna(0)
        df['segment_distance_m'] = _haversine_distance(df[latitude_col].shift(1).fillna(df[latitude_col]),
                                                     df[longitude_col].shift(1).fillna(df[longitude_col]),
                                                     df[latitude_col],
                                                     df[longitude_col])
        df['speed_knots'] = (df['segment_distance_m'] / df['time_diff_sec']).replace([np.inf, -np.inf], np.nan).fillna(0)
        df['course_deg'] = _calculate_initial_bearing(df[latitude_col].shift(1).fillna(df[latitude_col]),
                                                    df[longitude_col].shift(1).fillna(df[longitude_col]),
                                                    df[latitude_col],
                                                    df[longitude_col]).fillna(0)
        df['speed_change_knots'] = df['speed_knots'].diff().fillna(0)
        df['course_change_deg'] = df['course_deg'] - df['course_deg'].shift(1)
        df['course_change_deg'] = df['course_change_deg'].apply(lambda x: (x + 180) % 360 - 180 if pd.notna(x) else 0)
        df['course_change_deg'] = df['course_change_deg'].fillna(0)


    # Add more time-based features
    df['hour_of_day'] = df[timestamp_col].dt.hour
    df['day_of_week'] = df[timestamp_col].dt.dayofweek
    df['day_of_year'] = df[timestamp_col].dt.dayofyear
    df['month_of_year'] = df[timestamp_col].dt.month
    df['quarter'] = df[timestamp_col].dt.quarter
    df['is_weekend'] = df[timestamp_col].dt.dayofweek.isin([5, 6]).astype(int)

    print(f"Applied transformations. New columns: {list(df.columns)}")
    return df

if __name__ == '__main__':
    print("Running self-test for loading.py...")
    # Create a dummy parquet file with multiple vessels and some NaN/inf cases
    test_df = pd.DataFrame({
        'vessel_id': [1, 1, 1, 1, 2, 2, 2],
        'timestamp': pd.to_datetime([
            '2023-01-01 10:00:00', '2023-01-01 10:01:00', '2023-01-01 10:02:00', '2023-01-01 10:03:00',
            '2023-01-01 10:00:00', '2023-01-01 10:02:00', '2023-01-01 10:03:00'
        ]),
        'latitude': [40.0, 40.005, 40.010, 40.015, 30.0, 30.01, 30.02],
        'longitude': [-74.0, -74.005, -74.010, -74.015, -80.0, -80.01, -80.02],
        'value': [10, 20, 30, 40, 50, 60, 70] # Just some arbitrary feature
    })
    test_path = "test_loading_data.parquet"
    test_df.to_parquet(test_path, index=False)

    loaded_df = read_ts_data(test_path)
    print("Loaded DataFrame (first 5 rows):\n", loaded_df.head())

    transformed_df = apply_transformations(loaded_df.copy())
    print("\nTransformed DataFrame (first 5 rows):\n", transformed_df.head())
    print("\nTransformed DataFrame (last 5 rows):\n", transformed_df.tail())


    # Basic assertions
    assert 'time_diff_sec' in transformed_df.columns
    assert 'segment_distance_m' in transformed_df.columns
    assert 'speed_knots' in transformed_df.columns
    assert 'course_deg' in transformed_df.columns
    assert 'speed_change_knots' in transformed_df.columns
    assert 'course_change_deg' in transformed_df.columns
    assert 'hour_of_day' in transformed_df.columns
    assert 'is_weekend' in transformed_df.columns

    # Check for NaNs that should have been filled (e.g., first row of each group)
    assert transformed_df.loc[transformed_df['vessel_id'] == 1, 'time_diff_sec'].iloc[0] == 0
    assert transformed_df.loc[transformed_df['vessel_id'] == 2, 'time_diff_sec'].iloc[0] == 0
    assert transformed_df.loc[transformed_df['vessel_id'] == 1, 'segment_distance_m'].iloc[0] == 0
    assert transformed_df.loc[transformed_df['vessel_id'] == 2, 'segment_distance_m'].iloc[0] == 0
    assert transformed_df.loc[transformed_df['vessel_id'] == 1, 'speed_knots'].iloc[0] == 0
    assert transformed_df.loc[transformed_df['vessel_id'] == 2, 'speed_knots'].iloc[0] == 0
    assert transformed_df.loc[transformed_df['vessel_id'] == 1, 'course_deg'].iloc[0] == 0
    assert transformed_df.loc[transformed_df['vessel_id'] == 2, 'course_deg'].iloc[0] == 0
    assert transformed_df.loc[transformed_df['vessel_id'] == 1, 'speed_change_knots'].iloc[0] == 0
    assert transformed_df.loc[transformed_df['vessel_id'] == 2, 'speed_change_knots'].iloc[0] == 0
    assert transformed_df.loc[transformed_df['vessel_id'] == 1, 'course_change_deg'].iloc[0] == 0
    assert transformed_df.loc[transformed_df['vessel_id'] == 2, 'course_change_deg'].iloc[0] == 0

    print("\nloading.py self-test PASSED!")
    import os
    os.remove(test_path)

Writing loading.py


In [ ]:
!python loading.py

In [8]:
%%writefile data_loader.py
import os
import json
import shutil
import numpy as np
import pandas as pd
from pathlib import Path
from typing import Dict, Any, List, Union, Tuple, Optional
import hashlib
import traceback

from normalizer import Normalizer # Assuming this is a separate file you have

class RunConfig:
    """
    Stores the configuration relevant to data loading, including:
    - Features DataFrame (from formatter.py's output features.csv)
    - Normalization factors (calculated by dataloader.py)
    - CLI flags that affect data loading/preprocessing done by formatter.py and dataloader.py.
    """
    def __init__(self, cli_args: Dict[str, Any]):
        self.cli_args = cli_args
        self.features_df: Optional[pd.DataFrame] = None # From formatter.py's features.csv
        self.normalization_factors: Optional[Dict[str, Dict[str, float]]] = None # Calculated by DataLoader
        # Add other relevant cli_args to settings for hashing
        self.settings = {
            k: v for k, v in cli_args.items() if k not in ['cache_base_dir', 'features_csv_path', 'base_data_path', 'force_rebuild']
        }

    def to_hashable(self) -> str:
        """Generates a hashable string from the relevant configuration."""
        hash_input = {
            'settings': self.settings,
            # IMPORTANT: Convert DataFrame to a stable, hashable representation
            # For features_df, sorting by 'FeatureName' ensures consistent order
            'features': (self.features_df.sort_values('FeatureName').to_dict('records')
                         if self.features_df is not None else None),
            'normalization_factors': self.normalization_factors
        }
        # Use a more robust JSON dump for hashing (separators, sorted keys)
        hash_string = json.dumps(hash_input, sort_keys=True, separators=(',', ':')).encode('utf-8')
        return hashlib.md5(hash_string).hexdigest()

    def load_from_cache_dir(self, cache_dir: Path):
        """Loads features_df and normalization factors from the cache directory."""
        features_path = cache_dir / "features.csv"
        normalization_factors_path = cache_dir / "normalization_factors.json"

        if features_path.exists():
            self.features_df = pd.read_csv(features_path)
            # print(f"Loaded features_df from cache: {features_path}") # Removed verbose print
        else:
            print(f"Warning: features.csv not found in cache at {features_path}.")

        if normalization_factors_path.exists():
            with open(normalization_factors_path, 'r') as f:
                self.normalization_factors = json.load(f)
            # print(f"Loaded normalization factors from cache: {normalization_factors_path}") # Removed verbose print
        else:
            print(f"Warning: normalization_factors.json not found in cache at {normalization_factors_path}.")

    def save_to_cache_dir(self, cache_dir: Path):
        """Saves features_df and normalization factors to the cache directory."""
        cache_dir.mkdir(parents=True, exist_ok=True)

        if self.features_df is not None:
            self.features_df.to_csv(cache_dir / "features.csv", index=False)
            # print(f"Saved features_df to cache: {cache_dir / 'features.csv'}") # Removed verbose print

        if self.normalization_factors is not None:
            with open(cache_dir / "normalization_factors.json", 'w') as f:
                json.dump(self.normalization_factors, f, indent=4)
            # print(f"Saved normalization factors to cache: {cache_dir / 'normalization_factors.json'}") # Removed verbose print


class DataLoader:
    """
    Loads pre-processed data (X, Y numpy arrays) from formatter.py's output,
    calculates and applies normalization, and manages caching.
    """
    def __init__(self, cli_args: Dict[str, Any]):
        self.cli_args = cli_args
        self.base_data_path = Path(cli_args['base_data_path']) # Path to where formatter.py outputs (e.g., .../formatted_debug/)
        self.cache_base_dir = Path(cli_args['cache_base_dir'])
        self.time_gap = cli_args['time_gap'] # e.g., '30' for 30_min_time_gap
        self.is_data_chunked = cli_args.get('is_data_chunked', True) # formatter.py usually chunks data
        self.force_rebuild = cli_args.get('force_rebuild', False)
        self.normalization_method = cli_args.get('normalization_method', 'min_max')

        self.data: Dict[str, Tuple[np.ndarray, np.ndarray]] = {} # Stores (X, Y) numpy arrays (after normalization)
        self.run_config = RunConfig(cli_args) # Initialize RunConfig
        self.normalizer = Normalizer(method=self.normalization_method)

        self.cache_dir: Optional[Path] = None # Will be set once and reused

        # --- NEW LOGIC: Load features.csv and attempt to load normalization factors once at init ---
        # This ensures the RunConfig is fully populated before any hashing is done.
        features_csv_path = self.base_data_path / 'features.csv'
        if features_csv_path.exists():
            self.run_config.features_df = pd.read_csv(features_csv_path)
            print(f"DataLoader: Loaded features metadata from {features_csv_path}")
        else:
            raise FileNotFoundError(f"Error: features.csv not found at {features_csv_path}. "
                                    "Please ensure formatter.py has run successfully and outputs to base_data_path.")

        # Determine the cache directory hash based on initial config (including features_df)
        self.cache_dir = self._get_cache_dir() # Determine cache_dir once

        # Attempt to load normalization factors from this determined cache_dir
        # This is crucial for 'val' and 'test' to use the 'train' factors
        norm_factors_path = self.cache_dir / "normalization_factors.json"
        if norm_factors_path.exists():
            with open(norm_factors_path, 'r') as f:
                self.normalizer.normalization_factors = json.load(f)
                self.run_config.normalization_factors = self.normalizer.normalization_factors # Keep run_config in sync
            print(f"DataLoader: Loaded normalization factors from cache: {norm_factors_path}")
        else:
            print("DataLoader: Normalization factors not found in cache. Will calculate on first 'train' load.")

        print(f"DataLoader: Determined consistent cache directory: {self.cache_dir}")


    def _get_cache_dir(self) -> Path:
        """Determines and creates the cache directory based on RunConfig hash."""
        run_hash = self.run_config.to_hashable()
        cache_path = self.cache_base_dir / run_hash
        cache_path.mkdir(parents=True, exist_ok=True)
        return cache_path

    def _load_raw_formatter_output(self, dataset_type: str) -> Tuple[np.ndarray, Optional[np.ndarray]]:
        """
        Loads the unnormalized X and Y numpy arrays directly from formatter.py's output.
        """
        formatter_output_name_map = {
            'train': 'train_long_term_train',
            'val': 'valid_long_term_train', # Keep this as per your directory structure
            'test': 'test_long_term_test',
            'probe': 'probe_data' # Special case for probe data
        }

        if dataset_type == 'probe':
            probe_path = Path(self.cli_args.get('probe_data_path'))
            if not probe_path.exists():
                print(f"Warning: Probe data not found at {probe_path}. Returning empty arrays.")
                return (np.array([]), None) # Y is None for probe data
            X_probe = np.load(probe_path)
            return (X_probe, None)

        formatter_dir_name = formatter_output_name_map.get(dataset_type)
        if not formatter_dir_name:
            raise ValueError(f"Unknown dataset_type: {dataset_type}")

        base_data_set_path = self.base_data_path / formatter_dir_name
        x_dir = base_data_set_path / f"{self.time_gap}_min_time_gap_x"
        y_dir = base_data_set_path / f"{self.time_gap}_min_time_gap_y"

        if not x_dir.exists() or not y_dir.exists():
            print(f"    Warning: Data directories not found for {dataset_type} at {x_dir} and {y_dir}. Returning empty arrays.")
            return (np.array([]), np.array([]))

        X_parts = []
        Y_parts = []

        if self.is_data_chunked:
            x_files = sorted(x_dir.glob('*.npy'), key=lambda p: int(p.stem))
            y_files = sorted(y_dir.glob('*.npy'), key=lambda p: int(p.stem))

            if not x_files or not y_files:
                print(f"    Warning: No .npy chunks found in {x_dir} or {y_dir}. Returning empty arrays.")
                return (np.array([]), np.array([]))

            if len(x_files) != len(y_files):
                raise ValueError(f"Mismatch in number of X and Y chunks for {dataset_type} ({len(x_files)} vs {len(y_files)})")

            for x_file, y_file in zip(x_files, y_files):
                try:
                    X_parts.append(np.load(x_file))
                    y_array = np.load(y_file)
                    Y_parts.append(y_array)
                except Exception as e:
                    print(f"Error loading chunk {x_file} or {y_file}: {e}")
                    traceback.print_exc() # Print full traceback for debugging
                    continue # Skip problematic chunks for now

            if not X_parts:
                print(f"    Warning: No valid chunks loaded for {dataset_type}. Returning empty arrays.")
                return (np.array([]), np.array([]))

            X_raw = np.concatenate(X_parts, axis=0)
            Y_raw = np.concatenate(Y_parts, axis=0)

        else: # Load full files if not chunked
            x_file = x_dir.parent / f"{formatter_dir_name}_{self.time_gap}_min_time_gap_x.npy"
            y_file = y_dir.parent / f"{formatter_dir_name}_{self.time_gap}_min_time_gap_y.npy"
            if not x_file.exists() or not y_file.exists():
                    print(f"    Warning: Full data files not found for {dataset_type} at {x_file} and {y_file}. Returning empty arrays.")
                    return (np.array([]), np.array([]))
            try:
                X_raw = np.load(x_file)
                Y_raw = np.load(y_file)
            except Exception as e:
                print(f"Error loading full file {x_file} or {y_file}: {e}")
                traceback.print_exc() # Print full traceback for debugging
                return (np.array([]), np.array([]))

        print(f"    Loaded raw data for {dataset_type}: X shape {X_raw.shape}, Y shape {Y_raw.shape}")
        return X_raw, Y_raw

    def load_data(self, dataset_type: str, force_rebuild: Optional[bool] = None):
        """
        Loads, normalizes, and caches data for a given dataset type (train, val, test, probe).
        """
        if force_rebuild is None:
            force_rebuild = self.force_rebuild

        print(f"\nLoading data for '{dataset_type}' (force_rebuild: {force_rebuild})...")

        # Use the cache_dir that was determined at initialization
        cached_x_path = self.cache_dir / f"{dataset_type}_X.npy"
        cached_y_path = self.cache_dir / f"{dataset_type}_Y.npy"
        norm_factors_path_in_cache = self.cache_dir / "normalization_factors.json" # Redundant with self.normalizer, but good for saving

        # Attempt to load from cache
        if not force_rebuild and cached_x_path.exists() and (cached_y_path.exists() or dataset_type == 'probe') and norm_factors_path_in_cache.exists():
            try:
                print(f"    Attempting to load from cache: {cached_x_path}")
                X = np.load(cached_x_path)
                Y = np.load(cached_y_path) if dataset_type != 'probe' else None

                # Normalization factors are loaded at init, but re-verify
                if self.normalizer.normalization_factors is None:
                    # This should theoretically not happen if norm_factors_path_in_cache exists
                    with open(norm_factors_path_in_cache, 'r') as f:
                        self.normalizer.normalization_factors = json.load(f)
                        self.run_config.normalization_factors = self.normalizer.normalization_factors
                    print(f"    Loaded normalization factors (secondary check) from cache: {norm_factors_path_in_cache}")


                print(f"    Successfully loaded '{dataset_type}' data from cache.")
                self.data[dataset_type] = (X, Y)
                return

            except Exception as e:
                print(f"    Error loading from cache: {e}. Rebuilding data.")
                traceback.print_exc()
                # Clean up potentially corrupted cache files if an error occurs
                if cached_x_path.exists(): os.remove(cached_x_path)
                if cached_y_path.exists() and dataset_type != 'probe': os.remove(cached_y_path)
                if norm_factors_path_in_cache.exists(): os.remove(norm_factors_path_in_cache)


        # Load raw data from formatter output
        X_raw, Y_raw = self._load_raw_formatter_output(dataset_type)

        if X_raw.size == 0:
            print(f"    Raw data for '{dataset_type}' is empty. Skipping further processing.")
            self.data[dataset_type] = (np.array([]), np.array([]))
            return

        # Get feature names and their order from features.csv
        num_features_in_current_split = X_raw.shape[-1]

        # Use the features_df loaded during __init__
        if self.run_config.features_df is None:
            # This should have been caught in __init__
            raise RuntimeError("features_df is None. This indicates an issue during DataLoader initialization.")

        # Filter features based on what's actually present in X_raw and numeric_dtypes in features.csv
        numeric_features_df = self.run_config.features_df[
            self.run_config.features_df['FeatureName'].isin(self.run_config.features_df['FeatureName'].tolist()[:num_features_in_current_split]) &
            self.run_config.features_df['dtype'].isin(['float32', 'float64', 'int32', 'int64', 'int16', 'int8', 'uint8', 'bool'])
        ]
        features_to_normalize = numeric_features_df['FeatureName'].tolist()
        feature_names_in_X = self.run_config.features_df['FeatureName'].tolist()[:num_features_in_current_split]
        feature_mapping = {name: idx for idx, name in enumerate(feature_names_in_X)}

        excluded_from_normalization = ['mmsi', 'base_datetime', 'year', 'month', 'day_of_year', 'day_of_week', 'hour_of_day', 'quarter', 'is_weekend']
        features_to_normalize = [f for f in features_to_normalize if f not in excluded_from_normalization]

        # Calculate normalization factors ONLY if for 'train' data and factors are not already loaded/calculated
        if dataset_type == 'train' and self.normalizer.normalization_factors is None:
            X_raw_flat = X_raw.reshape(-1, X_raw.shape[-1])
            temp_df_for_factors = pd.DataFrame(X_raw_flat, columns=feature_names_in_X)

            print("    Calculating normalization factors from training data...")
            normalization_factors = self.normalizer.get_normalization_factors(temp_df_for_factors, features_to_normalize)
            self.normalizer.normalization_factors = normalization_factors
            self.run_config.normalization_factors = normalization_factors # Store in run_config for hashing consistency

            # Save normalization factors to the cache directory determined at init
            self.run_config.save_to_cache_dir(self.cache_dir)
            print(f"    Saved newly calculated normalization factors to cache: {norm_factors_path_in_cache}")
        elif self.normalizer.normalization_factors is None and dataset_type != 'train':
             # This means we are loading val/test before train, or norm factors failed to load from cache
             # This is a critical error, as val/test data needs to be normalized with train factors.
             raise RuntimeError(f"Normalization factors are not available for '{dataset_type}' data. "
                                "Please ensure 'train' data is loaded first to calculate/load factors, "
                                "or ensure normalization_factors.json exists in cache.")


        # Apply normalization using the loaded/calculated factors
        if self.normalizer.normalization_factors:
            print(f"    Applying {self.normalizer.method} normalization to '{dataset_type}' data...")
            X_normalized = self.normalizer.normalize_data(X_raw,
                                                            self.normalizer.normalization_factors,
                                                            feature_mapping)
            print(f"    Normalization applied to X data for '{dataset_type}'.")
            X = X_normalized
            # Y data (latitude, longitude) should typically NOT be normalized for prediction
            Y = Y_raw
            print(f"    Y data is kept unnormalized as it represents target coordinates.")
        else:
            print("    Normalization factors not available. Skipping normalization.")
            X = X_raw
            Y = Y_raw

        # Store processed data
        self.data[dataset_type] = (X, Y)

        # Cache the processed data (X, Y)
        if self.cache_dir:
            try:
                print(f"    Caching processed data for '{dataset_type}' to {self.cache_dir}...")
                np.save(cached_x_path, X)
                if dataset_type != 'probe':
                    np.save(cached_y_path, Y)
                print(f"    Successfully cached '{dataset_type}' data.")
            except Exception as e:
                print(f"    Error caching data for '{dataset_type}': {e}")


    def get_data(self, dataset_type: str) -> Tuple[np.ndarray, Optional[np.ndarray]]:
        """Retrieves loaded data for a given dataset type."""
        if dataset_type not in self.data:
            raise ValueError(f"Data for '{dataset_type}' not loaded yet. Call load_data() first.")
        return self.data[dataset_type]

    # Add a method to get the normalizer, e.g., for use in `predict.py`
    def get_normalizer(self) -> Normalizer:
        if self.normalizer.normalization_factors is None:
            raise RuntimeError("Normalization factors are not set. Load 'train' data first.")
        return self.normalizer

# --- Configuration for direct script execution ---
CLI_ARGS = {
    'cache_base_dir': '/content/data/new_york/dataloader_cache', # Example cache directory
    'base_data_path': '/content/data/new_york/formatted_debug',   # Path to formatter.py output
    'time_gap': '30', # Must match time_gaps in config.py and formatter.py
    'is_data_chunked': True, # True if formatter.py saved chunks
    'force_rebuild': False, # Set to True to ignore cache - generally false for cache
    'normalization_method': 'min_max', # Must match Normalizer initialization
    'input_sequence_length': 7, # Added for completeness as it affects RunConfig hash
    'output_sequence_length': 6, # Added for completeness
    'num_features': 51, # Added for completeness
    # 'probe_data_path': './temp_data_for_dl_test/probe_data.npy', # Example for probe data
}

# --- Main execution block ---
if __name__ == "__main__":
    print("Running data_loader.py as a script...")
    try:
        # Initialize and run the DataLoader
        data_loader = DataLoader(CLI_ARGS)

        # Force rebuild for this test run to ensure new cache generation if needed
        # In a real scenario, you'd typically set force_rebuild: False in CLI_ARGS
        # unless you change raw data or config.
        # However, for testing this fix, it's good to force it.

        data_loader.load_data(dataset_type='train', force_rebuild=True)
        data_loader.load_data(dataset_type='val', force_rebuild=True)
        data_loader.load_data(dataset_type='test', force_rebuild=True)

        print("\nDataLoader self-test PASSED!")
        X_train, Y_train = data_loader.get_data('train')
        X_val, Y_val = data_loader.get_data('val')
        X_test, Y_test = data_loader.get_data('test')

        print(f"Final loaded train shapes: X={X_train.shape}, Y={Y_train.shape}")
        print(f"Final loaded val shapes: X={X_val.shape}, Y={Y_val.shape}")
        print(f"Final loaded test shapes: X={X_test.shape}, Y={X_test.shape}") # Should be Y_test.shape

    except Exception as e:
        print(f"An error occurred during data loading: {e}")
        traceback.print_exc() # Print full traceback for debugging

Overwriting data_loader.py


In [12]:
!python data_loader.py

Running data_loader.py as a script...
DataLoader: Loaded features metadata from /content/data/new_york/formatted_debug/features.csv
DataLoader: Normalization factors not found in cache. Will calculate on first 'train' load.
DataLoader: Determined consistent cache directory: /content/data/new_york/dataloader_cache/cb5170adc6753993f2abbdd88ef376a9

Loading data for 'train' (force_rebuild: True)...
    Loaded raw data for train: X shape (1084072, 37, 51), Y shape (1084072, 6, 2)
    Calculating normalization factors from training data...
    Saved newly calculated normalization factors to cache: /content/data/new_york/dataloader_cache/cb5170adc6753993f2abbdd88ef376a9/normalization_factors.json
    Applying min_max normalization to 'train' data...
    Normalization applied to X data for 'train'.
    Y data is kept unnormalized as it represents target coordinates.
    Caching processed data for 'train' to /content/data/new_york/dataloader_cache/cb5170adc6753993f2abbdd88ef376a9...
    Succes

In [7]:
%%writefile generator.py
import numpy as np
from typing import Tuple, Generator

class DataGenerator:
    """
    A simple data generator for Keras/TensorFlow models.
    Assumes X and Y are already loaded and normalized by the DataLoader.
    """
    def __init__(self, X: np.ndarray, Y: np.ndarray, batch_size: int):
        self.X = X
        self.Y = Y
        self.batch_size = batch_size
        self.num_samples = X.shape[0]

    def __len__(self) -> int:
        """Denotes the number of batches per epoch."""
        return int(np.ceil(self.num_samples / self.batch_size))

    def generate(self) -> Generator[Tuple[np.ndarray, np.ndarray], None, None]:
        """Generates batches of data."""
        indices = np.arange(self.num_samples)
        np.random.shuffle(indices) # Shuffle data for each epoch

        for start_idx in range(0, self.num_samples, self.batch_size):
            batch_indices = indices[start_idx : start_idx + self.batch_size]

            batch_X = self.X[batch_indices]
            batch_Y = self.Y[batch_indices]

            yield batch_X, batch_Y

if __name__ == "__main__":
    print("Running self-test for data_generator.py...")

    # Create dummy normalized data (as if from DataLoader)
    dummy_X = np.random.rand(100, 7, 44).astype(np.float32) # 100 samples, 7 timesteps, 44 features
    dummy_Y = np.random.rand(100, 2).astype(np.float32)    # 100 samples, 2 prediction targets

    batch_size = 32
    generator = DataGenerator(dummy_X, dummy_Y, batch_size)

    # Test length
    expected_len = int(np.ceil(100 / 32))
    assert len(generator) == expected_len, f"Expected length {expected_len}, got {len(generator)}"
    print(f"Generator length: {len(generator)} batches.")

    # Test generation
    for i, (batch_X, batch_Y) in enumerate(generator.generate()):
        print(f"Batch {i+1}: X shape {batch_X.shape}, Y shape {batch_Y.shape}")
        assert batch_X.shape[0] <= batch_size
        assert batch_X.shape[1:] == (7, 44)
        assert batch_Y.shape[0] <= batch_size
        assert batch_Y.shape[1:] == (2,)

    print("\nDataGenerator self-test PASSED!")

Writing generator.py


In [ ]:
!python generator.py

In [6]:
!pip install tensorflow keras-tuner numpy pandas scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 1.6 MB/s eta 0:00:00


In [27]:
%%writefile metrics_losses.py
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.metrics import Metric
import numpy as np

# Constants for Haversine calculation
R_EARTH = 6371.0 # Radius of Earth in kilometers

def haversine_distance(lat1, lon1, lat2, lon2):
    """
    Calculate the haversine distance between two sets of (latitude, longitude) coordinates.
    This function is vectorized to handle batches of data.

    Args:
        lat1, lon1: TensorFlow tensors of shape (batch_size, sequence_length).
        lat2, lon2: TensorFlow tensors of shape (batch_size, sequence_length).

    Returns:
        A tensor of haversine distances in kilometers.
    """
    # Use TensorFlow's native math functions for consistency and reliability
    lat1_rad = tf.experimental.numpy.deg2rad(lat1)
    lon1_rad = tf.experimental.numpy.deg2rad(lon1)
    lat2_rad = tf.experimental.numpy.deg2rad(lat2)
    lon2_rad = tf.experimental.numpy.deg2rad(lon2)

    dlon = lon2_rad - lon1_rad
    dlat = lat2_rad - lat1_rad

    # Haversine formula implementation
    a = tf.sin(dlat / 2.0)**2 + tf.cos(lat1_rad) * tf.cos(lat2_rad) * tf.sin(dlon / 2.0)**2

    # Clip 'a' to ensure input to atan2 is in valid range [0, 1] due to
    # floating-point inaccuracies.
    a_clipped = tf.clip_by_value(a, 0.0, 1.0)
    c = 2.0 * tf.math.atan2(tf.sqrt(a_clipped), tf.sqrt(1.0 - a_clipped))

    distance = R_EARTH * c
    # Return mean distance, ensuring any residual nans are zeroed out
    distance = tf.where(tf.math.is_nan(distance), 0.0, distance)
    return distance

def haversine_loss(y_true, y_pred):
    """
    Calculates the mean haversine distance between predicted and true coordinates.
    The inputs are expected to have a shape of (batch_size, timesteps, 2),
    where the last dimension represents [latitude, longitude].
    """
    y_true_lat = y_true[..., 0]
    y_true_lon = y_true[..., 1]
    y_pred_lat = y_pred[..., 0]
    y_pred_lon = y_pred[..., 1]

    distance = haversine_distance(y_true_lat, y_true_lon, y_pred_lat, y_pred_lon)

    # Return the mean distance per batch
    return tf.reduce_mean(distance)


class HaversineMetric(Metric):
    """
    Custom Keras metric to compute the mean Haversine distance,
    with clipping safeguards for numerical stability.
    """
    def __init__(self, name="haversine_metric", **kwargs):
        super(HaversineMetric, self).__init__(name=name, **kwargs)
        self.total = self.add_weight(name="total", initializer="zeros")
        self.count = self.add_weight(name="count", initializer="zeros")

    def update_state(self, y_true, y_pred, sample_weight=None):
        """
        Updates the state with a new batch of predictions and true values.
        """
        y_true_lat = y_true[..., 0]
        y_true_lon = y_true[..., 1]
        y_pred_lat = y_pred[..., 0]
        y_pred_lon = y_pred[..., 1]

        # Calculate haversine distance for each sample
        distances = haversine_distance(y_true_lat, y_true_lon, y_pred_lat, y_pred_lon)

        # Update the state variables
        self.total.assign_add(tf.reduce_sum(distances))
        self.count.assign_add(tf.cast(tf.size(distances), dtype=tf.float32))

    def result(self):
        """
        Returns the mean Haversine distance over all processed batches.
        """
        return self.total / self.count

    def reset_state(self):
        """
        Resets the state of the metric.
        """
        self.total.assign(0.0)
        self.count.assign(0.0)

# --- Self-testing block for metrics_losses.py ---
if __name__ == "__main__":
    print("Running self-test for metrics_losses.py...")

    # Create mock true and predicted data
    y_true_mock = np.array([
        [[34.0522, -118.2437], [34.0522, -118.2437]],  # Los Angeles
        [[40.7128, -74.0060], [40.7128, -74.0060]]   # New York
    ], dtype=np.float32)

    # Test case 1: Identical predictions, distance should be close to 0
    y_pred_mock_1 = np.array([
        [[34.0522, -118.2437], [34.0522, -118.2437]],
        [[40.7128, -74.0060], [40.7128, -74.0060]]
    ], dtype=np.float32)

    # Test haversine_loss
    loss_1 = haversine_loss(y_true_mock, y_pred_mock_1)
    print(f"Loss with identical predictions: {loss_1.numpy():.4f} km (should be ~0)")
    assert np.isclose(loss_1.numpy(), 0.0), "Haversine loss with identical predictions failed."

    # Test HaversineMetric
    metric_1 = HaversineMetric()
    metric_1.update_state(y_true_mock, y_pred_mock_1)
    result_1 = metric_1.result()
    print(f"Metric with identical predictions: {result_1.numpy():.4f} km (should be ~0)")
    assert np.isclose(result_1.numpy(), 0.0), "Haversine metric with identical predictions failed."
    metric_1.reset_state()

    # Test case 2: Different predictions, distance should be non-zero
    y_pred_mock_2 = np.array([
        [[34.0522, -118.2437], [35.0522, -119.2437]],  # Los Angeles to slightly different location
        [[40.7128, -74.0060], [41.7128, -75.0060]]     # New York to slightly different location
    ], dtype=np.float32)

    # Test haversine_loss
    loss_2 = haversine_loss(y_true_mock, y_pred_mock_2)
    print(f"Loss with different predictions: {loss_2.numpy():.4f} km (should be > 0)")
    assert loss_2.numpy() > 0.0, "Haversine loss with different predictions failed."

    # Test HaversineMetric
    metric_2 = HaversineMetric()
    metric_2.update_state(y_true_mock, y_pred_mock_2)
    result_2 = metric_2.result()
    print(f"Metric with different predictions: {result_2.numpy():.4f} km (should be > 0)")
    assert result_2.numpy() > 0.0, "Haversine metric with different predictions failed."

    print("\nAll metrics_losses.py self-tests PASSED!")


Overwriting metrics_losses.py


In [28]:
!python metrics_losses.py

2025-08-04 18:56:05.153822: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754333765.174168   60628 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754333765.180352   60628 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Running self-test for metrics_losses.py...
2025-08-04 18:56:09.676041: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
I0000 00:00:1754333769.676188   60628 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13838 MB memory:  -> device: 0, name: Tesla T4, pci bus

In [ ]:
!pip install tensorflow keras-tuner numpy pandas scikit-learn

In [29]:
%%writefile models.py
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.layers import LSTM, GRU, Bidirectional, Dense, Dropout, Attention, Input, Reshape, Permute, Multiply, Concatenate, Add, Lambda
import tensorflow.keras.backend as K

# Assuming haversine_loss and HaversineMetric are defined elsewhere.
# For the purpose of this example and to make the self-test runnable,
# we'll define simple mock versions.
def haversine_loss(y_true, y_pred):
    # This is a placeholder. You should use your actual loss function here.
    return tf.reduce_mean(tf.square(y_true - y_pred))

class HaversineMetric(tf.keras.metrics.Metric):
    def __init__(self, name='haversine_metric', **kwargs):
        super(HaversineMetric, self).__init__(name=name, **kwargs)
        self.total = self.add_weight(name='total', initializer='zeros')
        self.count = self.add_weight(name='count', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        # This is a placeholder. You should use your actual metric logic here.
        values = tf.abs(y_true - y_pred)
        self.total.assign_add(tf.reduce_sum(values))
        self.count.assign_add(tf.cast(tf.size(y_true), dtype=tf.float32))

    def result(self):
        return self.total / self.count

    def reset_state(self):
        self.total.assign(0.0)
        self.count.assign(0.0)

def build_lstm_model(hp, input_shape, output_dim):
    """Builds an LSTM model with tunable hyperparameters."""
    model_input = Input(shape=input_shape)
    x = model_input

    # Layers: search from 1–5
    num_lstm_layers = hp.Int('num_lstm_layers', min_value=1, max_value=5, step=1)
    for i in range(num_lstm_layers):
        # LSTM units: use wider ranges (e.g., 32–512)
        lstm_units = hp.Int(f'lstm_units_{i}', min_value=32, max_value=512, step=32)
        return_sequences = True if i < num_lstm_layers - 1 else False
        x = LSTM(lstm_units, return_sequences=return_sequences)(x)
        # Dropout: 0.1–0.5
        x = Dropout(hp.Float(f'dropout_{i}', min_value=0.1, max_value=0.5, step=0.1))(x)
    #residual skip: project the input’s global summary into the same dim as x
    skip = Dense(K.int_shape(x)[-1])(
        Lambda(lambda z: K.mean(z, axis=1))(model_input)
    )
    x = Add()([x, skip])

    output_timesteps = output_dim // 2
    # The final Dense layer's output is now constrained to [-1, 1] using tanh.
    x = Dense(output_dim, activation='tanh')(x)
    # This Lambda layer scales the tanh output to the correct lat/lon ranges.
    # We create a scaling tensor and multiply the output by it.
    scaling_factors = tf.constant([90.0, 180.0] * output_timesteps, dtype=tf.float32)
    x = x * scaling_factors
    # The output is then reshaped to match the target shape (timesteps, 2)
    model_output = Reshape((output_timesteps, 2))(x)

    model = Model(inputs=model_input, outputs=model_output)

    learning_rate = hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, clipvalue=1.0)
    model.compile(optimizer=optimizer, loss=haversine_loss, metrics=[HaversineMetric()])

    return model

def build_gru_model(hp, input_shape, output_dim):
    """Builds a GRU model with tunable hyperparameters."""
    model_input = Input(shape=input_shape)
    x = model_input

    # Layers: search from 1–5
    num_gru_layers = hp.Int('num_gru_layers', min_value=1, max_value=5, step=1)
    for i in range(num_gru_layers):
        # GRU units: use wider ranges (e.g., 32–512)
        gru_units = hp.Int(f'gru_units_{i}', min_value=32, max_value=512, step=32)
        return_sequences = True if i < num_gru_layers - 1 else False
        x = GRU(gru_units, return_sequences=return_sequences)(x)
        # Dropout: 0.1–0.5
        x = Dropout(hp.Float(f'dropout_{i}', min_value=0.1, max_value=0.5, step=0.1))(x)
    #residual skip: project the input’s global summary into the same dim as x
    skip = Dense(K.int_shape(x)[-1])(
        Lambda(lambda z: K.mean(z, axis=1))(model_input)
    )
    x = Add()([x, skip])

    output_timesteps = output_dim // 2
    x = Dense(output_dim, activation='tanh')(x)
    scaling_factors = tf.constant([90.0, 180.0] * output_timesteps, dtype=tf.float32)
    x = x * scaling_factors
    model_output = Reshape((output_timesteps, 2))(x)

    model = Model(inputs=model_input, outputs=model_output)

    learning_rate = hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, clipvalue=1.0)
    model.compile(optimizer=optimizer, loss=haversine_loss, metrics=[HaversineMetric()])

    return model

def build_bilstm_model(hp, input_shape, output_dim):
    """Builds a Bidirectional LSTM model with tunable hyperparameters."""
    model_input = Input(shape=input_shape)
    x = model_input

    # Layers: search from 1–5
    num_bilstm_layers = hp.Int('num_bilstm_layers', min_value=1, max_value=5, step=1)
    for i in range(num_bilstm_layers):
        # BiLSTM units: use wider ranges (e.g., 32–512)
        bilstm_units = hp.Int(f'bilstm_units_{i}', min_value=32, max_value=512, step=32)
        return_sequences = True if i < num_bilstm_layers - 1 else False
        x = Bidirectional(LSTM(bilstm_units, return_sequences=return_sequences))(x)
        # Dropout: 0.1–0.5
        x = Dropout(hp.Float(f'dropout_{i}', min_value=0.1, max_value=0.5, step=0.1))(x)
    #residual skip: project the input’s global summary into the same dim as x
    skip = Dense(K.int_shape(x)[-1])(
        Lambda(lambda z: K.mean(z, axis=1))(model_input)
    )
    x = Add()([x, skip])

    output_timesteps = output_dim // 2
    x = Dense(output_dim, activation='tanh')(x)
    scaling_factors = tf.constant([90.0, 180.0] * output_timesteps, dtype=tf.float32)
    x = x * scaling_factors
    model_output = Reshape((output_timesteps, 2))(x)

    model = Model(inputs=model_input, outputs=model_output)

    learning_rate = hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, clipvalue=1.0)
    model.compile(optimizer=optimizer, loss=haversine_loss, metrics=[HaversineMetric()])

    return model

def build_bilstm_attention_model(hp, input_shape, output_dim):
    """Builds a Bidirectional LSTM model with Attention mechanism."""
    model_input = Input(shape=input_shape)
    x = model_input

    # Layers: search from 1–5
    num_bilstm_layers = hp.Int('num_bilstm_attention_layers', min_value=1, max_value=5, step=1)
    for i in range(num_bilstm_layers):
        # BiLSTM units: use wider ranges (e.g., 32–512)
        bilstm_units = hp.Int(f'bilstm_attention_units_{i}', min_value=32, max_value=512, step=32)
        # Always return sequences for attention layer
        x = Bidirectional(LSTM(bilstm_units, return_sequences=True))(x) # Output: (batch_size, timesteps, units*2)
        # Dropout: 0.1–0.5
        x = Dropout(hp.Float(f'dropout_attention_{i}', min_value=0.1, max_value=0.5, step=0.1))(x)

    # Attention Mechanism
    attention_scores = Dense(1, activation='tanh', name='attention_scores')(x)
    attention_weights = layers.Activation('softmax', name='attention_weights')(attention_scores)
    weighted_sequence = Multiply(name='weighted_sequence')([x, attention_weights])
    context_vector = layers.Lambda(lambda z: tf.reduce_sum(z, axis=1), name='context_vector')(weighted_sequence)

    # ── Residual skip: project the input’s global summary into the same dim as context_vector ──
    skip = Dense(K.int_shape(context_vector)[-1])(
        Lambda(lambda z: K.mean(z, axis=1))(model_input))
    merged = Add()([context_vector, skip])

    output_timesteps = output_dim // 2
    x = Dense(output_dim, activation='tanh')(merged)
    scaling_factors = tf.constant([90.0, 180.0] * output_timesteps, dtype=tf.float32)
    x = x * scaling_factors
    model_output = Reshape((output_timesteps, 2))(x)

    model = Model(inputs=model_input, outputs=model_output)

    learning_rate = hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, clipvalue=1.0)
    model.compile(optimizer=optimizer, loss=haversine_loss, metrics=[HaversineMetric()])

    return model


# --- Self-testing block for models.py ---
if __name__ == "__main__":
    print("Running self-test for models.py...")

    class MockHyperParameters:
        def Int(self, name, min_value, max_value, step):
            if 'units' in name: return 64
            if 'layers' in name: return 1
            if 'batch_size' in name: return 32
            return min_value
        def Float(self, name, min_value, max_value, step=None, sampling=None):
            if 'learning_rate' in name: return 0.001
            return 0.2
        def Choice(self, name, values, default=None):
            return values[0]

    hp = MockHyperParameters()
    input_shape = (10, 7)
    output_dim = 2 * 2

    # Test LSTM Model
    print("\nTesting LSTM Model...")
    lstm_model = build_lstm_model(hp, input_shape, output_dim)
    lstm_model.summary()
    assert lstm_model.output_shape == (None, output_dim // 2, 2), f"LSTM output shape mismatch: {lstm_model.output_shape}"
    print("LSTM Model test PASSED.")

    # Test GRU Model
    print("\nTesting GRU Model...")
    gru_model = build_gru_model(hp, input_shape, output_dim)
    gru_model.summary()
    assert gru_model.output_shape == (None, output_dim // 2, 2), f"GRU output shape mismatch: {gru_model.output_shape}"
    print("GRU Model test PASSED.")

    # Test BiLSTM Model
    print("\nTesting BiLSTM Model...")
    bilstm_model = build_bilstm_model(hp, input_shape, output_dim)
    bilstm_model.summary()
    assert bilstm_model.output_shape == (None, output_dim // 2, 2), f"BiLSTM output shape mismatch: {bilstm_model.output_shape}"
    print("BiLSTM Model test PASSED.")

    # Test BiLSTM with Attention Model
    print("\nTesting BiLSTM with Attention Model...")
    bilstm_attention_model = build_bilstm_attention_model(hp, input_shape, output_dim)
    bilstm_attention_model.summary()
    assert bilstm_attention_model.output_shape == (None, output_dim // 2, 2), f"BiLSTM Attention output shape mismatch: {bilstm_attention_model.output_shape}"
    print("BiLSTM with Attention Model test PASSED.")

    print("\nAll models.py self-tests PASSED!")


Overwriting models.py


In [30]:
!python models.py

2025-08-04 18:56:27.958372: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754333787.976908   60867 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754333787.982476   60867 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Running self-test for models.py...

Testing LSTM Model...
2025-08-04 18:56:32.158839: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
I0000 00:00:1754333792.158982   60867 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13838 MB memory:  -> device: 0, name: Te

In [31]:
%%writefile hyperparameter_tuner.py
# hyperparameter_tuner.py
import os
import json
import numpy as np
import pandas as pd
import tensorflow as tf
import keras_tuner as kt
from pathlib import Path
from typing import Dict, Any
from tensorflow.keras.callbacks import EarlyStopping

# Ensure these imports correctly point to your modules
from data_loader import DataLoader
from metrics_losses import haversine_loss, HaversineMetric
from models import (
    build_lstm_model,
    build_gru_model,
    build_bilstm_model,
    build_bilstm_attention_model
)

class HyperparamTuner:
    """
    Performs hyperparameter tuning for LSTM/GRU models using KerasTuner.
    It now dynamically computes sequence lengths and feature counts based on the
    shape of the loaded data, preventing shape mismatch errors.
    """
    def __init__(self, cli_args: Dict[str, Any]):
        self.cli_args = cli_args
        self.project_name = cli_args['tuning_project_name']
        self.log_dir = Path(cli_args['tuning_results_dir'])

        # NOTE: We no longer pre-calculate the shapes here.
        # Instead, we will determine them dynamically once the data is loaded.
        self.input_shape = None
        self.output_dim = None

        # Prepare logging directories
        self.log_dir.mkdir(parents=True, exist_ok=True)
        self.all_trials_csv = self.log_dir / f"{self.project_name}_all_trials.csv"
        self.best_params_json = self.log_dir / f"{self.project_name}_best_params.json"

        print(f"Tuner initialized for project: {self.project_name}")
        print(f"Results will be saved to: {self.log_dir}")
        print("Model input/output shapes will be determined from loaded data.")

    def model_builder(self, hp: kt.HyperParameters) -> tf.keras.Model:
        # Determine model type (fixed or sampled)
        model_type = (
            self.cli_args.get('fixed_model_type')
            if 'fixed_model_type' in self.cli_args
            else hp.Choice(
                'model_type', ['lstm', 'gru', 'bilstm', 'bilstm_attention'], default='bilstm_attention'
            )
        )

        # Build selected architecture
        if model_type == 'lstm':
            model = build_lstm_model(hp, self.input_shape, self.output_dim)
        elif model_type == 'gru':
            model = build_gru_model(hp, self.input_shape, self.output_dim)
        elif model_type == 'bilstm':
            model = build_bilstm_model(hp, self.input_shape, self.output_dim)
        else:
            model = build_bilstm_attention_model(hp, self.input_shape, self.output_dim)

        model.compile(
            optimizer=tf.keras.optimizers.Adam(
                learning_rate=hp.Float('learning_rate', 1e-5, 1e-2, sampling='LOG', default=1e-3)
            ),
            loss=haversine_loss,
            metrics=[HaversineMetric(name='haversine_metric')]
        )
        return model

    def run_tuning(self):
        # Load and normalize data
        print("Loading data for tuning...")
        dl = DataLoader(self.cli_args)
        dl.load_data('train')
        dl.load_data('val')
        X_train, Y_train = dl.get_data('train')
        X_val, Y_val     = dl.get_data('val')

        # Dynamically set the input and output shapes based on the loaded data
        self.input_shape = X_train.shape[1:]
        self.output_dim = Y_train.shape[1] * Y_train.shape[2]

        print(f"Training data shape: X={X_train.shape}, Y={Y_train.shape}")
        print(f"Validation data shape: X={X_val.shape}, Y={Y_val.shape}")

        # Final check for empty data after loading
        if X_train.shape[0] == 0 or X_val.shape[0] == 0:
            print("Error: Empty data. Aborting tuning.")
            return

        print(f"Tuner is now configured with Input Shape: {self.input_shape}")
        print(f"Tuner is now configured with Output Dimension: {self.output_dim}")

        all_models = ['lstm', 'gru', 'bilstm', 'bilstm_attention']
        all_records = []

        for arch in all_models:
            print(f"\n=== Tuning {arch} ===")
            self.cli_args['fixed_model_type'] = arch
            tuner = kt.RandomSearch(
                self.model_builder,
                objective=kt.Objective('val_haversine_metric', 'min'),
                max_trials=self.cli_args.get('trials_per_arch', 5),
                executions_per_trial=self.cli_args.get('executions_per_trial', 1),
                directory=str(self.log_dir),
                project_name=f"{self.project_name}_{arch}",
                overwrite=False
            )
            tuner.search(
                X_train, Y_train,
                epochs=self.cli_args.get('epochs', 10),
                validation_data=(X_val, Y_val),
                batch_size=self.cli_args.get('batch_size', 32),
                callbacks=[EarlyStopping(
                    monitor='val_haversine_metric',
                    patience=self.cli_args.get('patience', 5),
                    mode='min',
                    restore_best_weights=True
                )],
                verbose=1
            )

            # Collect trial results
            trials = tuner.oracle.get_best_trials(num_trials=len(tuner.oracle.trials))
            for t in trials:
                if t.score is not None:
                    all_records.append({
                        'model_type': arch,
                        'trial_id': t.trial_id,
                        **t.hyperparameters.values,
                        'val_haversine_metric': t.score
                    })
            best_hp = tuner.get_best_hyperparameters(num_trials=1)
            if best_hp:
                print(f"Best for {arch}: {best_hp[0].values}")

        if not all_records:
            print("No trials recorded. Exiting.")
            return

        df = pd.DataFrame(all_records).sort_values('val_haversine_metric')
        print("\nAll Trials Summary:")
        print(df.to_string(index=False))
        df.to_csv(self.all_trials_csv, index=False)
        print(f"Saved trials to {self.all_trials_csv}")

        # ── NEW: pick the best hyperparameters *per* architecture ──
        best_per_arch = {}
        for arch in all_models:
            arch_df = df[df['model_type'] == arch].sort_values('val_haversine_metric')
            if not arch_df.empty:
                row = arch_df.iloc[0]
                # drop the columns we don't need downstream
                params = row.drop(['model_type', 'trial_id', 'val_haversine_metric']).to_dict()
                best_per_arch[arch] = params

        # overwrite the JSON with a dict of {arch: best_params}
        with open(self.best_params_json, 'w') as f:
            json.dump(best_per_arch, f, indent=2)
        print(f"Saved best params for each arch to {self.best_params_json}")

        # Verify tuner directories
        print("--- Verify tuner directories ---")
        for arch in all_models:
            proj = self.log_dir / f"{self.project_name}_{arch}"
            if proj.is_dir():
                trials = [d for d in os.listdir(proj) if d.startswith('trial_')]
                print(f"{arch} trials: {trials}")
            else:
                print(f"Missing directory for {arch}: {proj}")
        print("--- Done ---")


if __name__=='__main__':
    args = {
        'base_data_path': '/content/data/new_york/formatted_debug',
        'cache_base_dir': '/content/data/new_york/dataloader_cache',
        'tuning_results_dir': './tuner_logs',
        'tuning_project_name': 'ship_pred_tuning',
        'time_gap': '30',
        'interpolation_time_gap_minutes': '5',
        #'num_features': 49, # This will be overridden dynamically
        'trials_per_arch': 5,
        'executions_per_trial': 1,
        'epochs': 15,
        #'batch_size': 16,
        'patience': 5
    }

    print("Initializing Hyperparameter Tuner...")
    tuner = HyperparamTuner(args)
    tuner.run_tuning()


Overwriting hyperparameter_tuner.py


In [23]:
# Zip the tuner_logs directory
!zip -r tuner_logs.zip tuner_logs

# Zip the dataloader_cache directory.
!zip -r dataloader_cache.zip /content/data/new_york/dataloader_cache

	zip warning: name not matched: tuner_logs

zip error: Nothing to do! (try: zip -r tuner_logs.zip . -i tuner_logs)
  adding: content/data/new_york/dataloader_cache/ (stored 0%)
  adding: content/data/new_york/dataloader_cache/cb5170adc6753993f2abbdd88ef376a9/ (stored 0%)
  adding: content/data/new_york/dataloader_cache/cb5170adc6753993f2abbdd88ef376a9/normalization_factors.json (deflated 92%)
  adding: content/data/new_york/dataloader_cache/cb5170adc6753993f2abbdd88ef376a9/val_Y.npy (deflated 78%)
  adding: content/data/new_york/dataloader_cache/cb5170adc6753993f2abbdd88ef376a9/train_Y.npy


zip error: Interrupted (aborting)


In [ ]:
!python hyperparameter_tuner.py

2025-08-04 18:56:43.454980: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754333803.476425   61009 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754333803.482752   61009 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Initializing Hyperparameter Tuner...
Tuner initialized for project: ship_pred_tuning
Results will be saved to: tuner_logs
Model input/output shapes will be determined from loaded data.
Loading data for tuning...
DataLoader: Loaded features metadata from /content/data/new_york/formatted_debug/features.csv
DataLoader: Loaded normalization factors from cache: /content/data/new_york/dataloader_cache/6b1eca834eee711d9ac8147edb69fcff/norma

In [ ]:
from google.colab import drive
import shutil, os

# 1️⃣ Mount Google Drive (remount if needed)
drive.mount('/content/drive', force_remount=True)

# 2️⃣ Define source folder and desired zip name
src_path = '/content/tuner_logs'
zip_name = 'tuner_logs'   # resulting archive will be zip_name.zip

# 3️⃣ Set destination directory in your Drive
dst_dir = '/content/drive/MyDrive'
os.makedirs(dst_dir, exist_ok=True)
dst_base = os.path.join(dst_dir, zip_name)

# 4️⃣ Create the zip archive
shutil.make_archive(base_name=dst_base, format='zip', root_dir=src_path)

print(f"✅ Zipped «{src_path}» → «{dst_base}.zip» in your MyDrive.")


In [ ]:
%%writefile train_and_evaluate.py
# train_and_evaluate.py - Final version with corrected model-building and loss logic
import os
import json
import numpy as np
import pandas as pd
import tensorflow as tf
from pathlib import Path
from typing import Dict, Any, List, Tuple
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import Input, LSTM, GRU, Dense, Bidirectional, Layer, Dropout
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K
import time # Import the time module

# Assume these are available in the working directory
from data_loader import DataLoader
from metrics_losses import haversine_loss, HaversineMetric, haversine_distance

# --- Corrected Model-Building Functions ---
# These functions now accept a dictionary `hps` and access values directly.

def build_lstm_model(hps: Dict[str, Any], input_shape: tuple, output_dim: int) -> tf.keras.Model:
    """Builds an LSTM model using hyperparameters from a dictionary."""
    model_input = Input(shape=input_shape)
    x = model_input

    num_lstm_layers = int(hps.get('num_lstm_layers', 1))
    for i in range(num_lstm_layers):
        lstm_units = int(hps.get(f'lstm_units_{i}', 32))
        dropout_rate = float(hps.get(f'dropout_{i}', 0.0))
        return_sequences = (i < num_lstm_layers - 1)
        x = LSTM(units=lstm_units, return_sequences=return_sequences)(x)
        if dropout_rate > 0:
            x = Dropout(dropout_rate)(x)

    model_output = Dense(output_dim, activation='linear')(x)
    model = Model(inputs=model_input, outputs=model_output)
    return model

def build_gru_model(hps: Dict[str, Any], input_shape: tuple, output_dim: int) -> tf.keras.Model:
    """Builds a GRU model using hyperparameters from a dictionary."""
    model_input = Input(shape=input_shape)
    x = model_input

    num_gru_layers = int(hps.get('num_gru_layers', 1))
    for i in range(num_gru_layers):
        gru_units = int(hps.get(f'gru_units_{i}', 32))
        dropout_rate = float(hps.get(f'dropout_{i}', 0.0))
        return_sequences = (i < num_gru_layers - 1)
        x = GRU(units=gru_units, return_sequences=return_sequences)(x)
        if dropout_rate > 0:
            x = Dropout(dropout_rate)(x)

    model_output = Dense(output_dim, activation='linear')(x)
    model = Model(inputs=model_input, outputs=model_output)
    return model

def build_bilstm_model(hps: Dict[str, Any], input_shape: tuple, output_dim: int) -> tf.keras.Model:
    """Builds a Bidirectional LSTM model using hyperparameters from a dictionary."""
    model_input = Input(shape=input_shape)
    x = model_input

    num_bilstm_layers = int(hps.get('num_bilstm_layers', 1))
    for i in range(num_bilstm_layers):
        bilstm_units = int(hps.get(f'bilstm_units_{i}', 32))
        dropout_rate = float(hps.get(f'dropout_{i}', 0.0))
        return_sequences = (i < num_bilstm_layers - 1)
        x = Bidirectional(LSTM(units=bilstm_units, return_sequences=return_sequences))(x)
        if dropout_rate > 0:
            x = Dropout(dropout_rate)(x)

    model_output = Dense(output_dim, activation='linear')(x)
    model = Model(inputs=model_input, outputs=model_output)
    return model

class AttentionLayer(Layer):
    """
    Custom attention layer for the BiLSTM with Attention model.
    """
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def build(self, input_shape):
        self.W = self.add_weight(name="att_weight", shape=(input_shape[-1], input_shape[-1]),
                                     initializer="glorot_uniform", trainable=True)
        self.b = self.add_weight(name="att_bias", shape=(input_shape[-1],),
                                     initializer="zeros", trainable=True)
        self.u = self.add_weight(name="att_context", shape=(input_shape[-1],),
                                     initializer="glorot_uniform", trainable=True)
        super().build(input_shape)

    def call(self, x):
        u_prime = K.tanh(K.dot(x, self.W) + self.b)
        alpha = K.softmax(K.dot(u_prime, K.expand_dims(self.u)))
        output = K.sum(x * alpha, axis=1)
        return output

    def get_config(self):
        config = super().get_config()
        return config

def build_bilstm_attention_model(hps: Dict[str, Any], input_shape: tuple, output_dim: int) -> tf.keras.Model:
    """Builds a Bidirectional LSTM with Attention model using hyperparameters from a dictionary."""
    model_input = Input(shape=input_shape)
    x = model_input

    num_bilstm_layers = int(hps.get('num_bilstm_attention_layers', 1))
    for i in range(num_bilstm_layers):
        bilstm_units = int(hps.get(f'bilstm_attention_units_{i}', 32))
        return_sequences = (i < num_bilstm_layers - 1) or True # The last BiLSTM must return sequences for the attention layer
        x = Bidirectional(LSTM(units=bilstm_units, return_sequences=return_sequences))(x)
        dropout_rate = float(hps.get(f'dropout_attention_{i}', 0.0))
        if dropout_rate > 0:
            x = Dropout(dropout_rate)(x)

    attention_output = AttentionLayer()(x)
    model_output = Dense(output_dim, activation='linear')(attention_output)
    model = Model(inputs=model_input, outputs=model_output)
    return model

# Custom objects dictionary for loading models
custom_objects = {
    'haversine_loss': haversine_loss,
    'HaversineMetric': HaversineMetric,
    'AttentionLayer': AttentionLayer,
}


class TrainAndEvaluateAllModels:
    """
    A class to train and evaluate all model types (LSTM, GRU, BiLSTM, BiLSTM_Attention)
    using their individually best hyperparameters found by the hyperparameter tuner.
    """
    def __init__(self, cli_args: Dict[str, Any]):
        self.cli_args = cli_args

        # Define paths for loading and saving
        self.tuning_results_dir = Path(cli_args['tuning_results_dir'])
        self.best_params_path = self.tuning_results_dir / f"{cli_args['tuning_project_name']}_best_params.json"

        self.model_save_dir = Path(cli_args['model_save_dir'])
        self.model_save_dir.mkdir(parents=True, exist_ok=True)

        self.evaluation_report_dir = Path(cli_args['evaluation_report_dir'])
        self.evaluation_report_dir.mkdir(parents=True, exist_ok=True)

        # Calculate input_sequence_length based on 3 hours history and interpolation_time_gap
        length_of_history_seconds = 3 * 60 * 60 # 3 hours
        # The time_gap is now an integer, so we can use it directly
        time_gap_seconds = self.cli_args['time_gap'] * 60
        calculated_input_sequence_length = int(length_of_history_seconds / time_gap_seconds) + 1
        self.cli_args['input_sequence_length'] = calculated_input_sequence_length

        # input_shape: (timesteps_history, num_features)
        self.input_shape = (
            self.cli_args['input_sequence_length'],
            self.cli_args['num_features']
        )
        # output_dim: total predicted values (timesteps * 2 for lat/lon)
        self.output_dim = self.cli_args['output_sequence_length'] * 2

        self.model_types = ['lstm', 'gru', 'bilstm', 'bilstm_attention']

        print("Training and Evaluation initialized to run for all model types.")
        print(f"Loading best hyperparameters per model from: {self.best_params_path}")
        print(f"Trained models will be saved to: {self.model_save_dir}")
        print(f"Evaluation reports will be saved to: {self.evaluation_report_dir}")

    def load_best_hyperparameters_per_model(self) -> Dict[str, Any]:
        try:
            with open(self.best_params_path, 'r') as f:
                best_hps_per_model = json.load(f)
            # ─── Inject model_type into each hps dict ───
            for model_type, hps in best_hps_per_model.items():
                hps['model_type'] = model_type
            print(f"Loaded best hyperparameters from {self.best_params_path}")
            return best_hps_per_model
        except FileNotFoundError:
            print(f"ERROR: Best-params JSON not found at {self.best_params_path}")
            raise

    def load_and_prepare_data(self) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
        """
        Loads and returns the train, validation, and test datasets.
        """
        print("Loading data for training and evaluation...")
        # The DataLoader requires a 'time_gap' key in cli_args, which is a string.
        # This has been fixed to an integer, so we must convert it back to a string for DataLoader.
        # Note: The original override has been removed, so this uses the correct `time_gap` value.
        self.cli_args['time_gap'] = str(self.cli_args['time_gap'])

        dl = DataLoader(self.cli_args)
        dl.load_data('train')
        dl.load_data('val')
        dl.load_data('test')

        X_train, Y_train = dl.get_data('train')
        X_val, Y_val = dl.get_data('val')
        X_test, Y_test = dl.get_data('test')

        print(f"Training data shape: X={X_train.shape}, Y={Y_train.shape}")
        print(f"Validation data shape: X={X_val.shape}, Y={Y_val.shape}")
        print(f"Test data shape: X={X_test.shape}, Y={Y_test.shape}")

        return X_train, Y_train, X_val, Y_val, X_test, Y_test

    def build_model(self, hps: Dict[str, Any]) -> tf.keras.Model:
        """Builds a Keras model using the provided hyperparameters."""
        model_type = hps.get('model_type')
        print(f"Building {model_type} model with best hyperparameters...")

        if model_type == 'lstm':
            model = build_lstm_model(hps, self.input_shape, self.output_dim)
        elif model_type == 'gru':
            model = build_gru_model(hps, self.input_shape, self.output_dim)
        elif model_type == 'bilstm':
            model = build_bilstm_model(hps, self.input_shape, self.output_dim)
        elif model_type == 'bilstm_attention':
            model = build_bilstm_attention_model(hps, self.input_shape, self.output_dim)
        else:
            raise ValueError(f"Unknown model type: {model_type}")

        model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=hps.get('learning_rate', 1e-3)),
            loss=haversine_loss,
            metrics=[HaversineMetric(name='haversine_metric')]
        )
        return model

    def train_model(self, model: tf.keras.Model, X_train: np.ndarray, Y_train: np.ndarray, X_val: np.ndarray, Y_val: np.ndarray, hps: Dict[str, Any]) -> None:
        """Trains the model with early stopping."""
        model_type = hps['model_type']
        print(f"Training {model_type} model...")
        early_stopping = EarlyStopping(
            monitor='val_haversine_metric',
            patience=int(hps.get('patience', 10)),
            mode='min',
            restore_best_weights=True
        )
        reduce_lr = ReduceLROnPlateau(
             monitor='val_haversine_metric',
             factor=0.5,
             patience=5,
             min_lr=1e-6,
             verbose=1
        )

        start_time = time.time() # Start time measurement
        model.fit(
            X_train, Y_train,
            epochs=int(hps.get('epochs', 100)),
            validation_data=(X_val, Y_val),
            batch_size=int(hps.get('batch_size', 32)),
            callbacks=[early_stopping, reduce_lr],
            verbose=1
        )
        end_time = time.time() # End time measurement
        training_time = end_time - start_time
        print(f"{model_type} model training finished in {training_time:.2f} seconds.")

        # Store training time and parameter count in the HPS dictionary or a separate results dictionary
        hps['training_time_seconds'] = training_time
        hps['num_parameters'] = model.count_params() # Capture parameter count here
        print(f"Model Parameters: {hps['num_parameters']}")

        # Save the trained model
        model_path = self.model_save_dir / f"{model_type}_best_model.h5"
        model.save(model_path)
        print(f"Trained model saved to: {model_path}")

        # Capture model file size after saving
        hps['model_file_size_bytes'] = os.path.getsize(model_path)
        print(f"Model File Size: {hps['model_file_size_bytes']} bytes")


    def evaluate_metrics(self, model: tf.keras.Model, X_test: np.ndarray, Y_test: np.ndarray, model_type: str, training_time: float, num_parameters: int, model_file_size_bytes: int) -> Dict[str, Any]:
        """
        Performs a full evaluation on the test set, calculating all the metrics
        specified in the paper.
        """
        print(f"\nEvaluating {model_type} model on test data...")

        y_pred = model.predict(X_test, verbose=1)

        Y_test_reshaped = Y_test.reshape(Y_test.shape[0], -1, 2)
        y_pred_reshaped = y_pred.reshape(y_pred.shape[0], -1, 2)

        num_timesteps = Y_test_reshaped.shape[1]

        # We now use the correct `time_gap` from the cli_args dictionary
        # This value is now an integer, as requested
        time_gap_minutes = self.cli_args.get('time_gap', 30)

        evaluation_results = {
            "model_type": model_type,
            "training_time_seconds": training_time,
            "num_parameters": num_parameters, # Add parameter count here
            "model_file_size_bytes": model_file_size_bytes # Add file size here
        }

        # --- Haversine-based Metrics ---
        all_haversine_errors = haversine_distance(Y_test_reshaped, y_pred_reshaped)

        # Correctly calculate metrics for 1h, 2h, and 3h horizons
        # Forecasts are at 30, 60, 90, 120, 150, 180 minutes
        # Index 1 corresponds to 60 min (1h)
        # Index 3 corresponds to 120 min (2h)
        # Index 5 corresponds to 180 min (3h)

        # Mean Haversine error across all predictions
        evaluation_results["mean_haversine_error_overall_km"] = float(np.mean(all_haversine_errors))
        evaluation_results["std_haversine_error_overall_km"] = float(np.std(all_haversine_errors))
        evaluation_results["median_haversine_error_overall_km"] = float(np.median(all_haversine_errors))
        evaluation_results["max_haversine_error_overall_km"] = float(np.max(all_haversine_errors))

        # RMSE Haversine
        evaluation_results["rmse_haversine_overall_km"] = float(np.sqrt(np.mean(np.square(all_haversine_errors))))


        # Per-horizon Haversine errors
        for i in range(num_timesteps):
            horizon_minutes = (i + 1) * time_gap_minutes
            horizon_error = all_haversine_errors[:, i]
            evaluation_results[f"mean_haversine_error_{horizon_minutes}min_km"] = float(np.mean(horizon_error))
            evaluation_results[f"std_haversine_error_{horizon_minutes}min_km"] = float(np.std(horizon_error))
            evaluation_results[f"median_haversine_error_{horizon_minutes}min_km"] = float(np.median(horizon_error))
            evaluation_results[f"max_haversine_error_{horizon_minutes}min_km"] = float(np.max(horizon_error))
            evaluation_results[f"rmse_haversine_{horizon_minutes}min_km"] = float(np.sqrt(np.mean(np.square(horizon_error))))

        # Specific 1h, 2h, 3h horizons if they exist
        if time_gap_minutes > 0: # Avoid division by zero
            if (60 / time_gap_minutes) - 1 < num_timesteps and (60 % time_gap_minutes == 0): # Check if 1h horizon exists
                idx_1h = (60 // time_gap_minutes) - 1
                horizon_error_1h = all_haversine_errors[:, idx_1h]
                evaluation_results["mean_haversine_error_1h_km"] = float(np.mean(horizon_error_1h))
                evaluation_results["rmse_haversine_1h_km"] = float(np.sqrt(np.mean(np.square(horizon_error_1h))))

            if (120 / time_gap_minutes) - 1 < num_timesteps and (120 % time_gap_minutes == 0): # Check if 2h horizon exists
                idx_2h = (120 // time_gap_minutes) - 1
                horizon_error_2h = all_haversine_errors[:, idx_2h]
                evaluation_results["mean_haversine_error_2h_km"] = float(np.mean(horizon_error_2h))
                evaluation_results["rmse_haversine_2h_km"] = float(np.sqrt(np.mean(np.square(horizon_error_2h))))

            if (180 / time_gap_minutes) - 1 < num_timesteps and (180 % time_gap_minutes == 0): # Check if 3h horizon exists
                idx_3h = (180 // time_gap_minutes) - 1
                horizon_error_3h = all_haversine_errors[:, idx_3h]
                evaluation_results["mean_haversine_error_3h_km"] = float(np.mean(horizon_error_3h))
                evaluation_results["rmse_haversine_3h_km"] = float(np.sqrt(np.mean(np.square(horizon_error_3h))))


        # --- Coordinate-based Metrics (using Lat/Lon directly) ---
        y_test_flat = Y_test_reshaped.reshape(-1, 2)
        y_pred_flat = y_pred_reshaped.reshape(-1, 2)

        lat_true = y_test_flat[:, 0]
        lat_pred = y_pred_flat[:, 0]
        lon_true = y_test_flat[:, 1]
        lon_pred = y_pred_flat[:, 1]

        # Latitude Metrics
        evaluation_results["lat_mae_deg"] = float(mean_absolute_error(lat_true, lat_pred))
        evaluation_results["lat_mse_deg"] = float(mean_squared_error(lat_true, lat_pred))
        evaluation_results["lat_rmse_deg"] = float(np.sqrt(evaluation_results["lat_mse_deg"]))
        evaluation_results["lat_r2"] = float(r2_score(lat_true, lat_pred))
        evaluation_results["lat_median_abs_error_deg"] = float(np.median(np.abs(lat_true - lat_pred)))
        evaluation_results["lat_max_abs_error_deg"] = float(np.max(np.abs(lat_true - lat_pred)))

        # Longitude Metrics
        evaluation_results["lon_mae_deg"] = float(mean_absolute_error(lon_true, lon_pred))
        evaluation_results["lon_mse_deg"] = float(mean_squared_error(lon_true, lon_pred))
        evaluation_results["lon_rmse_deg"] = float(np.sqrt(evaluation_results["lon_mse_deg"]))
        evaluation_results["lon_r2"] = float(r2_score(lon_true, lon_pred))
        evaluation_results["lon_median_abs_error_deg"] = float(np.median(np.abs(lon_true - lon_pred)))
        evaluation_results["lon_max_abs_error_deg"] = float(np.max(np.abs(lon_true - lon_pred)))

        # Average Coordinate Metrics
        evaluation_results["avg_mae_deg"] = float((evaluation_results["lat_mae_deg"] + evaluation_results["lon_mae_deg"]) / 2)
        evaluation_results["avg_mse_deg"] = float((evaluation_results["lat_mse_deg"] + evaluation_results["lon_mse_deg"]) / 2)
        evaluation_results["avg_rmse_deg"] = float((evaluation_results["lat_rmse_deg"] + evaluation_results["lon_rmse_deg"]) / 2)
        evaluation_results["avg_r2"] = float((evaluation_results["lat_r2"] + evaluation_results["lon_r2"]) / 2)
        evaluation_results["avg_median_abs_error_deg"] = float((evaluation_results["lat_median_abs_error_deg"] + evaluation_results["lon_median_abs_error_deg"]) / 2)
        evaluation_results["avg_max_abs_error_deg"] = float((evaluation_results["lat_max_abs_error_deg"] + evaluation_results["lon_max_abs_error_deg"]) / 2)


        print(f"\n--- {model_type.upper()} Evaluation Complete ---")
        for k, v in evaluation_results.items():
            print(f"{k:<40}: {v:.6f}") # Increased padding for better alignment

        return evaluation_results

    def run(self):
        """Orchestrates the entire training and evaluation process for all models."""
        try:
            best_hps_per_model = self.load_best_hyperparameters_per_model()
            X_train, Y_train, X_val, Y_val, X_test, Y_test = self.load_and_prepare_data()

            for model_type, hps in best_hps_per_model.items():
                print(f"\n--- Starting process for model: {model_type} ---")
                model = self.build_model(hps)

                # train_model now stores training time, parameter count, and model file size in hps
                self.train_model(model, X_train, Y_train, X_val, Y_val, hps)

                # Use a fresh instance of the model with restored weights for evaluation
                best_model_path = self.model_save_dir / f"{model_type}_best_model.h5"
                best_model = tf.keras.models.load_model(best_model_path, custom_objects=custom_objects)

                # Pass the training time, num_parameters, and model_file_size_bytes to evaluate_metrics
                evaluation_results = self.evaluate_metrics(
                    best_model,
                    X_test,
                    Y_test,
                    model_type,
                    hps.get('training_time_seconds', 0.0),
                    hps.get('num_parameters', 0), # Pass num_parameters
                    hps.get('model_file_size_bytes', 0) # Pass model_file_size_bytes
                )

                # Save the final evaluation report to a JSON file
                evaluation_report_path = self.evaluation_report_dir / f"{model_type}_evaluation_report.json"
                with open(evaluation_report_path, 'w') as f:
                    json.dump(evaluation_results, f, indent=4)
                print(f"\nEvaluation report saved to: {evaluation_report_path}")
                print(f"--- Process for model {model_type} completed. ---\n")


        except Exception as e:
            print(f"An error occurred during training or evaluation: {e}")
            raise


if __name__ == '__main__':
    real_data_args = {
        'base_data_path': '/content/data/new_york/formatted_debug',
        # Fixed typo in the cache directory path
        'cache_base_dir': '/content/data/new_york/dataloader_cache',
        'tuning_results_dir': '/content/tuner_logs',
        # FIX: Changed the tuning_project_name to match the user's provided file path.
        'tuning_project_name': 'ship_pred_tuning',
        'model_save_dir': './trained_models',
        'evaluation_report_dir': './evaluation_reports',
        'output_sequence_length': 6,
        'num_features': 49,
        # Changed time_gap to an integer as requested
        'time_gap': 30,
        'epochs': 10,
        'batch_size': 16,
        'patience': 4,
    }

    print("--- Initializing Training and Evaluation Script for All Models ---")
    trainer = TrainAndEvaluateAllModels(real_data_args)
    print("\n--- Running Training and Evaluation for All Models ---")
    trainer.run()
    print("Training and evaluation process for all models completed.")

In [ ]:
!python train_and_evaluate.py

In [ ]:
import os
import json
import pandas as pd
from IPython.display import display
from typing import List, Dict, Any

def benchmark_models():
    """
    Reads the evaluation reports for each model and presents a benchmark
    of their most significant performance metrics in a concise table.
    """
    # Define the directory where the evaluation reports are saved
    evaluation_report_dir = "./evaluation_reports"

    # List of models to benchmark
    models = ["lstm", "gru", "bilstm", "bilstm_attention"]

    benchmark_data: List[Dict[str, Any]] = []

    print("--- Starting Model Performance Benchmark ---")

    # Iterate through each model to load its evaluation report
    for model_name in models:
        report_file_path = os.path.join(evaluation_report_dir, f"{model_name}_evaluation_report.json")

        # Check if the evaluation report file exists
        if not os.path.exists(report_file_path):
            print(f"Warning: Evaluation report for {model_name} not found at {report_file_path}")
            continue

        try:
            with open(report_file_path, 'r') as f:
                report_data = json.load(f)

            # Extract only the key metrics for a less wide table
            metrics = {
                "Model": model_name,
                "Median Haversine Error (m)": report_data.get("median_haversine_error", "N/A"),
                "Mean Haversine Error (1h)": report_data.get("mean_haversine_error_1h", "N/A"),
                "Avg RMSE (deg)": report_data.get("avg_rmse_deg", "N/A"),
                "Avg MSE (deg)": report_data.get("avg_mse_deg", "N/A"),
                "Avg R2 Score": report_data.get("avg_r2", "N/A")
            }
            benchmark_data.append(metrics)

        except json.JSONDecodeError:
            print(f"Error: Could not decode JSON from file {report_file_path}")
            continue

    # Create a pandas DataFrame from the collected data
    if benchmark_data:
        benchmark_df = pd.DataFrame(benchmark_data)
        benchmark_df.set_index("Model", inplace=True)

        print("\n--- Model Performance Comparison (Key Metrics) ---")
        # Display the DataFrame
        display(benchmark_df)
    else:
        print("\nNo evaluation reports were found. Please ensure the training and evaluation scripts ran successfully.")

# Run the benchmark function
if __name__ == "__main__":
    benchmark_models()


In [ ]:
import json
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np

def plot_haversine_error_line_chart(report_dir: str, models: list):
    """
    Generates a line chart comparing the mean Haversine error for all models
    at 1, 2, and 3-hour forecast horizons.

    Args:
        report_dir (str): The directory where the evaluation reports are stored.
        models (list): A list of model names (e.g., ['lstm', 'gru']).
    """
    model_data = {}
    print(f"Reading evaluation reports from directory: {report_dir}")

    # Loop through each model to load its corresponding evaluation report
    for model_name in models:
        report_path = Path(report_dir) / f"{model_name}_evaluation_report.json"

        if not report_path.exists():
            print(f"Warning: Report not found for model '{model_name}' at {report_path}. Skipping.")
            continue

        try:
            with open(report_path, 'r') as f:
                report_data = json.load(f)

            # Extract the specific metrics we want to plot.
            errors = [
                report_data.get("mean_haversine_error_1h"),
                report_data.get("mean_haversine_error_2h"),
                report_data.get("mean_haversine_error_3h")
            ]
            if all(e is not None for e in errors):
                model_data[model_name.upper()] = errors
                print(f"Successfully loaded '{model_name}' data.")
            else:
                print(f"Warning: Missing Haversine error data for '{model_name}'.")

        except json.JSONDecodeError:
            print(f"Error: Could not decode JSON from {report_path}.")

    if not model_data:
        print("No valid error data found to plot. Exiting.")
        return

    # --- Create the line chart ---
    plt.style.use('seaborn-v0_8-whitegrid')

    forecast_horizons = [1, 2, 3]  # Horizons in hours

    fig, ax = plt.subplots(figsize=(12, 7))

    # Define a color palette for the lines
    colors = plt.cm.tab10.colors

    # Plot lines for each model
    for i, (model_name, errors) in enumerate(model_data.items()):
        ax.plot(forecast_horizons, errors, marker='o', linestyle='-', label=model_name, color=colors[i])

        # Annotate each point on the line
        for x, y in zip(forecast_horizons, errors):
            ax.annotate(f'{y:.0f}', (x, y), textcoords="offset points", xytext=(0, 10), ha='center', fontsize=8)

    # Add labels and title for clarity
    ax.set_title('Mean Haversine Error Trend by Forecast Horizon', fontsize=16, fontweight='bold')
    ax.set_xlabel('Forecast Horizon (hours)', fontsize=12)
    ax.set_ylabel('Mean Haversine Error (km)', fontsize=12)
    ax.set_xticks(forecast_horizons)
    ax.set_xticklabels([f'{h}-Hour' for h in forecast_horizons])
    ax.legend(title='Model Architecture')

    plt.tight_layout()
    plt.show()

if __name__ == '__main__':
    all_model_types = ['lstm', 'gru', 'bilstm', 'bilstm_attention']
    evaluation_report_directory = 'evaluation_reports'
    plot_haversine_error_line_chart(evaluation_report_directory, all_model_types)


In [ ]:
import os
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
from typing import Dict, Any, List

def load_evaluation_data(report_dir: Path) -> List[Dict[str, Any]]:
    """
    Loads all JSON evaluation reports from a specified directory into a list.
    """
    evaluation_data = []
    if not report_dir.is_dir():
        print(f"Error: Directory '{report_dir}' not found.")
        return evaluation_data

    # Find all JSON files that end with '_evaluation_report.json'
    json_files = sorted([f for f in report_dir.glob("*_evaluation_report.json")])
    if not json_files:
        print(f"No evaluation report files found in '{report_dir}'. Please ensure they are present.")
        return evaluation_data

    for file_path in json_files:
        try:
            with open(file_path, 'r') as f:
                data = json.load(f)
                # Extract model type from the filename
                model_type = file_path.name.replace('_evaluation_report.json', '')
                data['model_type'] = model_type  # Add model type for identification
                evaluation_data.append(data)
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON from file: {file_path}. Error: {e}")

    return evaluation_data

def plot_mse_over_time_haversine_style(evaluation_reports: List[Dict[str, Any]]):
    """
    Generates a line graph of the average MSE over different forecast horizons,
    styled to match the provided Haversine plot example.
    """
    if not evaluation_reports:
        return

    # Convert the list of dictionaries to a pandas DataFrame for easier plotting
    df = pd.DataFrame(evaluation_reports)
    df.set_index('model_type', inplace=True)
    df.index = df.index.str.upper()

    print("--- Generating Average MSE Trend Line Graph (Haversine Style) ---")

    # Dynamically find all columns that contain 'mse_'
    mse_metrics = [col for col in df.columns if 'mse_' in col]

    if not mse_metrics:
        print("Error: No columns with 'mse_' found in the evaluation data. Please check your JSON report keys.")
        return

    # --- Create the line chart with seaborn style and annotations ---
    plt.style.use('seaborn-v0_8-whitegrid')
    fig, ax = plt.subplots(figsize=(12, 7))

    # Transpose the data to have forecast horizons on the x-axis
    mse_data = df[mse_metrics].T

    # Create clearer labels for the x-axis from the metric names
    x_labels = [metric.replace('mse_', '').replace('_deg', '') for metric in mse_metrics]
    forecast_horizons = np.arange(len(x_labels))

    # Define a color palette for the lines
    colors = plt.cm.tab10.colors

    # Plot lines for each model
    for i, model_name in enumerate(mse_data.columns):
        errors = mse_data[model_name].values
        ax.plot(forecast_horizons, errors, marker='o', linestyle='-', label=model_name, color=colors[i])

        # Annotate each point on the line with the MSE value
        for x, y in zip(forecast_horizons, errors):
            ax.annotate(f'{y:.2f}', (x, y), textcoords="offset points", xytext=(0, 10), ha='center', fontsize=9)

    ax.set_title('Mean Squared Error Trend by Forecast Horizon', fontsize=16, fontweight='bold')
    ax.set_ylabel('Mean Squared Error (Degrees squared)', fontsize=12)
    ax.set_xlabel('Forecast Horizon', fontsize=12)
    ax.set_xticks(forecast_horizons)
    ax.set_xticklabels([f'{h}' for h in x_labels])
    ax.legend(title='Model Architecture')
    ax.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()

if __name__ == '__main__':
    # Define the directory where your evaluation reports are saved
    evaluation_reports_directory = Path('/content/evaluation_reports')

    # Load the data and plot the graph
    reports = load_evaluation_data(evaluation_reports_directory)
    if reports:
        plot_mse_over_time_haversine_style(reports)

    print("Plotting complete.")


In [ ]:
import json
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np

def plot_median_vs_max_haversine(report_dir: str, models: list):
    """
    Generates a grouped bar chart comparing the median and max Haversine error
    for all models.

    Args:
        report_dir (str): The directory where the evaluation reports are stored.
        models (list): A list of model names (e.g., ['lstm', 'gru']).
    """
    model_data = {}
    print(f"Reading evaluation reports from directory: {report_dir}")

    for model_name in models:
        report_path = Path(report_dir) / f"{model_name}_evaluation_report.json"

        if not report_path.exists():
            print(f"Warning: Report not found for model '{model_name}' at {report_path}. Skipping.")
            continue

        try:
            with open(report_path, 'r') as f:
                report_data = json.load(f)

            median_error = report_data.get("median_haversine_error")
            max_error = report_data.get("max_haversine_error")

            if median_error is not None and max_error is not None:
                model_data[model_name.upper()] = [median_error, max_error]
                print(f"Successfully loaded '{model_name}' median and max error data.")
            else:
                print(f"Warning: Missing median or max Haversine error data for '{model_name}'.")
        except json.JSONDecodeError:
            print(f"Error: Could not decode JSON from {report_path}.")

    if not model_data:
        print("No valid error data found to plot. Exiting.")
        return

    # --- Create the grouped bar chart ---
    plt.style.use('seaborn-v0_8-whitegrid')

    error_types = ['Median Error', 'Max Error']
    x = np.arange(len(error_types))
    width = 0.2

    fig, ax = plt.subplots(figsize=(12, 7))

    for i, (model_name, errors) in enumerate(model_data.items()):
        bars = ax.bar(x + i*width - width*1.5, errors, width, label=model_name)
        for bar in bars:
            height = bar.get_height()
            ax.annotate(f'{height:.0f}',
                        xy=(bar.get_x() + bar.get_width() / 2, height),
                        xytext=(0, 3),
                        textcoords="offset points",
                        ha='center', va='bottom', fontsize=8, rotation=90)

    ax.set_title('Median vs. Max Haversine Error', fontsize=16, fontweight='bold')
    ax.set_xlabel('Error Type', fontsize=12)
    ax.set_ylabel('Haversine Error (km)', fontsize=12)
    ax.set_xticks(x)
    ax.set_xticklabels(error_types)
    ax.legend()

    plt.tight_layout()
    plt.show()

if __name__ == '__main__':
    all_model_types = ['lstm', 'gru', 'bilstm', 'bilstm_attention']
    evaluation_report_directory = 'evaluation_reports'
    plot_median_vs_max_haversine(evaluation_report_directory, all_model_types)


In [ ]:
import json
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np

def plot_lat_lon_rmse(report_dir: str, models: list):
    """
    Generates a grouped bar chart comparing the Root Mean Square Error (RMSE)
    for latitude and longitude predictions for all models.

    Args:
        report_dir (str): The directory where the evaluation reports are stored.
        models (list): A list of model names (e.g., ['lstm', 'gru']).
    """
    model_data = {}
    print(f"Reading evaluation reports from directory: {report_dir}")

    for model_name in models:
        report_path = Path(report_dir) / f"{model_name}_evaluation_report.json"

        if not report_path.exists():
            print(f"Warning: Report not found for model '{model_name}' at {report_path}. Skipping.")
            continue

        try:
            with open(report_path, 'r') as f:
                report_data = json.load(f)

            lat_rmse = report_data.get("lat_rmse_deg")
            lon_rmse = report_data.get("lon_rmse_deg")

            if lat_rmse is not None and lon_rmse is not None:
                model_data[model_name.upper()] = [lat_rmse, lon_rmse]
                print(f"Successfully loaded '{model_name}' latitude and longitude RMSE data.")
            else:
                print(f"Warning: Missing latitude or longitude RMSE data for '{model_name}'.")

        except json.JSONDecodeError:
            print(f"Error: Could not decode JSON from {report_path}.")

    if not model_data:
        print("No valid error data found to plot. Exiting.")
        return

    # --- Create the grouped bar chart ---
    plt.style.use('seaborn-v0_8-whitegrid')

    error_types = ['Latitude RMSE', 'Longitude RMSE']
    x = np.arange(len(error_types))
    width = 0.2

    fig, ax = plt.subplots(figsize=(12, 7))

    for i, (model_name, errors) in enumerate(model_data.items()):
        bars = ax.bar(x + i*width - width*1.5, errors, width, label=model_name)
        for bar in bars:
            height = bar.get_height()
            ax.annotate(f'{height:.2f}',
                        xy=(bar.get_x() + bar.get_width() / 2, height),
                        xytext=(0, 3),
                        textcoords="offset points",
                        ha='center', va='bottom', fontsize=8, rotation=90)

    ax.set_title('Latitude vs. Longitude RMSE', fontsize=16, fontweight='bold')
    ax.set_xlabel('Error Type', fontsize=12)
    ax.set_ylabel('RMSE (degrees)', fontsize=12)
    ax.set_xticks(x)
    ax.set_xticklabels(error_types)
    ax.legend()

    plt.tight_layout()
    plt.show()

if __name__ == '__main__':
    all_model_types = ['lstm', 'gru', 'bilstm', 'bilstm_attention']
    evaluation_report_directory = 'evaluation_reports'
    plot_lat_lon_rmse(evaluation_report_directory, all_model_types)


In [ ]:
import os
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
from typing import Dict, Any, List

def load_evaluation_data(report_dir: Path) -> List[Dict[str, Any]]:
    """
    Loads all JSON evaluation reports from a specified directory into a list.
    """
    evaluation_data = []
    if not report_dir.is_dir():
        print(f"Error: Directory '{report_dir}' not found.")
        return evaluation_data

    # Find all JSON files that end with '_evaluation_report.json'
    json_files = sorted([f for f in report_dir.glob("*_evaluation_report.json")])
    if not json_files:
        print(f"No evaluation report files found in '{report_dir}'. Please run train_and_evaluate.py first.")
        return evaluation_data

    for file_path in json_files:
        try:
            with open(file_path, 'r') as f:
                data = json.load(f)
                # Extract model type from the filename
                model_type = file_path.name.replace('_evaluation_report.json', '')
                data['model_type'] = model_type  # Add model type for identification
                evaluation_data.append(data)
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON from file: {file_path}. Error: {e}")

    return evaluation_data

def plot_model_performance(evaluation_reports: List[Dict[str, Any]]):
    """
    Generates and displays performance plots for the models.
    """
    if not evaluation_reports:
        return

    # Convert the list of dictionaries to a pandas DataFrame for easier plotting
    df = pd.DataFrame(evaluation_reports)
    df.set_index('model_type', inplace=True)
    df.index = df.index.str.upper()

    print("--- Generating Performance Plots ---")

    # --- Plot 1: Mean Haversine Error Over Time ---
    fig, ax = plt.subplots(figsize=(10, 6))

    haversine_metrics = ['mean_haversine_error_1h', 'mean_haversine_error_2h', 'mean_haversine_error_3h']
    haversine_data = df[haversine_metrics].T

    haversine_data.plot(kind='bar', ax=ax, width=0.8)

    ax.set_title('Mean Haversine Error Over Time', fontsize=16)
    ax.set_ylabel('Error (meters)', fontsize=12)
    ax.set_xlabel('Forecast Horizon', fontsize=12)
    ax.set_xticklabels(['1-hour', '2-hour', '3-hour'], rotation=0)
    ax.legend(title='Model', bbox_to_anchor=(1.05, 1), loc='upper left')
    ax.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()


    # --- Plot 2: Average MSE and RMSE ---
    fig, ax = plt.subplots(figsize=(10, 6))

    mse_rmse_metrics = ['avg_mse_deg', 'avg_rmse_deg']

    df[mse_rmse_metrics].plot(kind='bar', ax=ax, width=0.8)

    ax.set_title('Average MSE and RMSE', fontsize=16)
    ax.set_ylabel('Error (Degrees)', fontsize=12)
    ax.set_xlabel('Model Type', fontsize=12)
    ax.set_xticklabels(df.index, rotation=0)
    ax.legend(title='Metric', bbox_to_anchor=(1.05, 1), loc='upper left')
    ax.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()


    # --- Plot 3: Average R-squared Score ---
    fig, ax = plt.subplots(figsize=(10, 6))

    df['avg_r2'].plot(kind='bar', ax=ax, color='teal', width=0.6)

    ax.set_title('Average R-squared (R²) Score', fontsize=16)
    ax.set_ylabel('R² Score', fontsize=12)
    ax.set_xlabel('Model Type', fontsize=12)
    ax.set_xticklabels(df.index, rotation=0)
    ax.axhline(0, color='grey', linestyle='--')
    ax.grid(axis='y', linestyle='--', alpha=0.7)
    ax.set_ylim(bottom=0)  # Ensure the bar chart starts from 0 for fair comparison

    # Add R² values on top of the bars
    for container in ax.containers:
        ax.bar_label(container, fmt='%.3f', padding=5)

    plt.tight_layout()
    plt.show()


if __name__ == '__main__':
    # Define the directory where your evaluation reports are saved
    evaluation_reports_directory = Path('/content/evaluation_reports')

    # Load the data and plot the graphs
    reports = load_evaluation_data(evaluation_reports_directory)
    if reports:
        plot_model_performance(reports)

    print("Plotting complete.")
